# Exploration des Données - Online Retail II

**Auteur:** Projet Marketing Analytics  
**Date:** 2024  
**Objectif:** Analyse exploratoire du dataset Online Retail II pour comprendre les patterns de vente et le comportement client

---

## 1. Introduction et Contexte du Projet

### 1.1 Contexte Business

Ce projet vise à créer une application d'aide à la décision marketing basée sur l'analyse des données transactionnelles d'un retailer en ligne. L'objectif est de fournir des insights actionnables pour optimiser les stratégies d'acquisition, de fidélisation et de valorisation client.

### 1.2 Objectifs de l'Analyse

- **Comprendre la structure des données** : identifier les variables clés et leur qualité
- **Analyser les patterns de vente** : tendances temporelles, saisonnalité, volumes
- **Profiler les clients** : comportements d'achat, segmentation naturelle
- **Identifier les opportunités** : segments à fort potentiel, produits clés
- **Préparer les données** : nettoyage et transformation pour les analyses avancées

### 1.3 Questions d'Analyse

1. Quelle est la qualité des données (valeurs manquantes, doublons, aberrations) ?
2. Comment évoluent les ventes dans le temps (tendance, saisonnalité) ?
3. Quelle est la distribution géographique des clients et du revenu ?
4. Quels sont les produits les plus vendus ?
5. Quel est le comportement d'achat typique (fréquence, montant) ?
6. Y a-t-il des segments de clients naturellement distincts ?
7. Quel est le taux d'annulation et son impact ?
8. Comment préparer les données pour l'analyse de cohortes et RFM ?

---

## 2. Imports et Configuration

In [ ]:
# Imports standard
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuration
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
import config

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Options d'affichage pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Imports réalisés avec succès")
print(f"Version pandas: {pd.__version__}")
print(f"Version numpy: {np.__version__}")

## 3. Chargement et Aperçu des Données

### 3.1 Chargement du Dataset

In [ ]:
# Chargement du dataset
# Le fichier utilise le point-virgule comme séparateur et la virgule comme séparateur décimal
df = pd.read_csv(
    '../data/raw/online_retail_II.csv',
    sep=';',
    decimal=',',
    encoding='utf-8',
    parse_dates=['InvoiceDate'],
    dayfirst=True
)

# Informations de base
print(f"✅ Dataset chargé avec succès")
print(f"Nombre de lignes: {len(df):,}")
print(f"Nombre de colonnes: {len(df.columns)}")
print(f"Mémoire utilisée: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nPériode des données:")
print(f"  - Date minimale: {df['InvoiceDate'].min()}")
print(f"  - Date maximale: {df['InvoiceDate'].max()}")
print(f"  - Durée: {(df['InvoiceDate'].max() - df['InvoiceDate'].min()).days} jours")

### 3.2 Structure du Dataset

In [ ]:
# Affichage des premières lignes
df.head(10)

In [ ]:
# Informations détaillées sur les types de données
print("=== INFORMATIONS SUR LE DATASET ===\n")
df.info()

print("\n=== TYPES DE DONNÉES ===")
print(df.dtypes)

print("\n=== STATISTIQUES DESCRIPTIVES ===")
df.describe(include='all')

## 3. Dictionnaire des Variables

Création d'un dictionnaire détaillé décrivant chaque variable du dataset.

In [ ]:
# Création du dictionnaire des variables
data_dict = {
    'Variable': ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country'],
    'Type': ['object (string)', 'object (string)', 'object (string)', 'int64', 'datetime64', 'float64', 'float64', 'object (string)'],
    'Sémantique': [
        'Numéro de facture unique',
        'Code produit unique',
        'Nom/description du produit',
        'Quantité de produits achetés',
        'Date et heure de la transaction',
        'Prix unitaire du produit',
        'Identifiant client unique',
        'Pays de résidence du client'
    ],
    'Unités/Valeurs': [
        'Alphanumérique, "C" préfixe = annulation',
        'Alphanumérique',
        'Texte libre',
        'Entier positif (peut être négatif pour annulations)',
        'Format: JJ/MM/AAAA HH:MM',
        'Livres Sterling (£)',
        'Numérique entier',
        'Nom de pays en anglais'
    ],
    'Exemple': [
        '536365, C536365',
        '85123A, 71053',
        'WHITE HANGING HEART T-LIGHT HOLDER',
        '6, -6',
        '01/12/2010 08:26',
        '2.55, 3.39',
        '17850.0',
        'United Kingdom, France'
    ],
    'Observations': [
        'Clé pour identifier les transactions',
        'Permet de tracker les produits',
        'Peut contenir des valeurs manquantes',
        'Négatif indique une annulation',
        'Pas de valeurs manquantes attendues',
        'Peut être 0 ou négatif (à vérifier)',
        'Nombreuses valeurs manquantes possibles',
        'Dimension géographique clé'
    ]
}

dict_df = pd.DataFrame(data_dict)

# Affichage stylisé
print("="*100)
print("DICTIONNAIRE DES VARIABLES - ONLINE RETAIL II".center(100))
print("="*100)
display(dict_df.style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-weight', 'bold'), ('text-align', 'center')]},
    {'selector': 'td', 'props': [('padding', '10px')]}
]))

print("\n\n=== VARIABLES À CRÉER POUR L'ANALYSE ===")
print("""
1. TotalAmount: Quantity × Price (montant total de la ligne de transaction)
2. InvoiceMonth: Mois extrait de InvoiceDate (pour agrégations temporelles)
3. InvoiceYear: Année extraite de InvoiceDate
4. InvoiceYearMonth: Combinaison Année-Mois
5. CohortMonth: Mois de première transaction du client (pour analyse de cohortes)
6. CohortIndex: Nombre de mois depuis la première transaction du client
7. IsCancellation: Booléen indiquant si la facture est une annulation (Invoice commence par 'C')
8. DayOfWeek: Jour de la semaine de la transaction
9. Hour: Heure de la transaction
""")

## 4. Analyse de la Qualité des Données

Cette section examine en détail la qualité des données pour identifier les problèmes potentiels avant l'analyse.

### 4.1 Valeurs Manquantes

In [ ]:
# Analyse des valeurs manquantes
print("="*80)
print("ANALYSE DES VALEURS MANQUANTES".center(80))
print("="*80)

# Calcul des statistiques
missing_count = df.isnull().sum()
missing_pct = (missing_count / len(df)) * 100
total_cells = df.shape[0] * df.shape[1]
total_missing = missing_count.sum()

# Création du DataFrame récapitulatif
missing_df = pd.DataFrame({
    'Colonne': df.columns,
    'Valeurs manquantes': missing_count.values,
    'Pourcentage (%)': missing_pct.values,
    'Valeurs présentes': len(df) - missing_count.values
})
missing_df = missing_df.sort_values('Valeurs manquantes', ascending=False)

print(f"\nRécapitulatif global:")
print(f"  - Total de cellules: {total_cells:,}")
print(f"  - Cellules manquantes: {total_missing:,}")
print(f"  - Taux de données manquantes: {(total_missing/total_cells)*100:.2f}%")
print(f"\n" + "-"*80)

# Affichage du tableau
display(missing_df.style.background_gradient(subset=['Pourcentage (%)'], cmap='Reds'))

print("\n" + "="*80)
print("IMPACT BUSINESS DES VALEURS MANQUANTES".center(80))
print("="*80)

# Analyse par colonne
for col in missing_df[missing_df['Valeurs manquantes'] > 0]['Colonne'].values:
    pct = missing_df[missing_df['Colonne'] == col]['Pourcentage (%)'].values[0]
    count = missing_df[missing_df['Colonne'] == col]['Valeurs manquantes'].values[0]
    print(f"\n{col}:")
    print(f"  ➤ {count:,} valeurs manquantes ({pct:.2f}%)")
    
    if col == 'Customer ID':
        print(f"  ➤ CRITIQUE: Empêche l'analyse client (cohortes, RFM, CLV)")
        print(f"  ➤ Ces transactions sont probablement des ventes sans compte client")
        print(f"  ➤ Décision: EXCLURE ces lignes pour l'analyse client-centrique")
    elif col == 'Description':
        print(f"  ➤ MODÉRÉ: Rend difficile l'analyse produit")
        print(f"  ➤ Peut-être des produits spéciaux ou frais (ex: livraison)")
        print(f"  ➤ Décision: CONSERVER mais marquer comme 'Description Inconnue'")
    else:
        print(f"  ➤ Impact à évaluer selon le contexte")

print("\n" + "="*80)

In [ ]:
# Visualisation des valeurs manquantes
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Graphique 1: Nombre de valeurs manquantes par colonne
missing_to_plot = missing_df[missing_df['Valeurs manquantes'] > 0].sort_values('Valeurs manquantes')
axes[0].barh(missing_to_plot['Colonne'], missing_to_plot['Valeurs manquantes'], color='#E74C3C')
axes[0].set_xlabel('Nombre de valeurs manquantes', fontsize=12, fontweight='bold')
axes[0].set_title('Valeurs manquantes par colonne (nombre absolu)', fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)
for i, v in enumerate(missing_to_plot['Valeurs manquantes']):
    axes[0].text(v, i, f' {v:,}', va='center', fontsize=10)

# Graphique 2: Pourcentage de valeurs manquantes
axes[1].barh(missing_to_plot['Colonne'], missing_to_plot['Pourcentage (%)'], color='#E67E22')
axes[1].set_xlabel('Pourcentage de valeurs manquantes (%)', fontsize=12, fontweight='bold')
axes[1].set_title('Valeurs manquantes par colonne (%)', fontsize=14, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)
for i, v in enumerate(missing_to_plot['Pourcentage (%)']):
    axes[1].text(v, i, f' {v:.2f}%', va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\n📊 Interprétation:")
print("  - Les colonnes avec valeurs manquantes nécessitent une attention particulière")
print("  - Customer ID est crucial pour l'analyse client et doit être traité en priorité")
print("  - Description peut affecter l'analyse produit mais n'est pas bloquante")

### 4.2 Doublons

Analyse des lignes dupliquées complètes et partielles.

In [ ]:
print("="*80)
print("ANALYSE DES DOUBLONS".center(80))
print("="*80)

# 1. Doublons complets (toutes les colonnes identiques)
duplicates_full = df.duplicated().sum()
duplicates_full_pct = (duplicates_full / len(df)) * 100

print(f"\n1. DOUBLONS COMPLETS:")
print(f"   - Nombre de lignes en double: {duplicates_full:,}")
print(f"   - Pourcentage: {duplicates_full_pct:.2f}%")

if duplicates_full > 0:
    print(f"\n   Exemple de doublons complets:")
    duplicate_rows = df[df.duplicated(keep=False)].sort_values(['Invoice', 'StockCode'])
    display(duplicate_rows.head(10))
else:
    print(f"   ✅ Aucun doublon complet détecté")

# 2. Doublons partiels (même Invoice + StockCode)
print(f"\n2. DOUBLONS PARTIELS (même Invoice + StockCode):")
duplicates_partial = df.duplicated(subset=['Invoice', 'StockCode'], keep=False).sum()
duplicates_partial_unique = df.duplicated(subset=['Invoice', 'StockCode'], keep='first').sum()
duplicates_partial_pct = (duplicates_partial / len(df)) * 100

print(f"   - Lignes concernées: {duplicates_partial:,}")
print(f"   - Doublons à retirer (keep='first'): {duplicates_partial_unique:,}")
print(f"   - Pourcentage: {duplicates_partial_pct:.2f}%")

if duplicates_partial > 0:
    print(f"\n   Exemple de doublons partiels:")
    partial_dup_rows = df[df.duplicated(subset=['Invoice', 'StockCode'], keep=False)].sort_values(['Invoice', 'StockCode'])
    display(partial_dup_rows.head(10))
    
    # Analyse des différences
    print(f"\n   Analyse des différences dans les doublons partiels:")
    sample_invoice = partial_dup_rows.iloc[0]['Invoice']
    sample_stockcode = partial_dup_rows.iloc[0]['StockCode']
    sample_dups = df[(df['Invoice'] == sample_invoice) & (df['StockCode'] == sample_stockcode)]
    display(sample_dups)

# 3. Statistiques sur les factures
print(f"\n3. STATISTIQUES PAR FACTURE:")
invoice_stats = df.groupby('Invoice').agg({
    'StockCode': 'count',
    'Customer ID': 'first'
}).rename(columns={'StockCode': 'NbArticles'})

print(f"   - Nombre total de factures: {df['Invoice'].nunique():,}")
print(f"   - Nombre moyen d'articles par facture: {invoice_stats['NbArticles'].mean():.2f}")
print(f"   - Nombre médian d'articles par facture: {invoice_stats['NbArticles'].median():.0f}")
print(f"   - Maximum d'articles dans une facture: {invoice_stats['NbArticles'].max():,}")

print("\n" + "="*80)
print("DÉCISION SUR LE TRAITEMENT DES DOUBLONS".center(80))
print("="*80)
print("""
➤ Doublons complets: 
  - Si présents, les supprimer car ils n'apportent aucune information supplémentaire
  - Utiliser df.drop_duplicates()

➤ Doublons partiels (même Invoice + StockCode mais autres colonnes différentes):
  - À ANALYSER AU CAS PAR CAS
  - Peuvent être légitimes si Quantity ou Price diffèrent (corrections, ajustements)
  - Peuvent être des erreurs de saisie
  - Recommandation: Inspecter manuellement et décider selon le contexte business
""")

### 4.3 Règles Métier et Incohérences

Vérification de la cohérence des données par rapport aux règles business.

In [ ]:
print("="*80)
print("ANALYSE DES RÈGLES MÉTIER ET INCOHÉRENCES".center(80))
print("="*80)

# 1. Factures d'annulation (Invoice commençant par 'C')
print("\n1. FACTURES D'ANNULATION (Invoice avec 'C'):")
df['IsCancellation'] = df['Invoice'].astype(str).str.startswith('C')
cancellations = df['IsCancellation'].sum()
cancellation_rate = (cancellations / len(df)) * 100

print(f"   - Nombre de lignes d'annulation: {cancellations:,}")
print(f"   - Taux d'annulation: {cancellation_rate:.2f}%")
print(f"   - Nombre de factures d'annulation: {df[df['IsCancellation']]['Invoice'].nunique():,}")

# Calcul de l'impact financier des annulations (si possible)
if 'Price' in df.columns and 'Quantity' in df.columns:
    df_temp = df.copy()
    df_temp['TotalAmount'] = df_temp['Quantity'] * df_temp['Price']
    total_cancelled = df_temp[df_temp['IsCancellation']]['TotalAmount'].sum()
    total_revenue = df_temp[~df_temp['IsCancellation']]['TotalAmount'].sum()
    print(f"   - Montant total annulé: £{abs(total_cancelled):,.2f}")
    print(f"   - Montant total des ventes: £{total_revenue:,.2f}")
    print(f"   - Impact: {(abs(total_cancelled)/total_revenue)*100:.2f}% du CA")

# 2. Quantités négatives
print(f"\n2. QUANTITÉS NÉGATIVES:")
qty_negative = (df['Quantity'] < 0).sum()
qty_negative_pct = (qty_negative / len(df)) * 100
qty_zero = (df['Quantity'] == 0).sum()
qty_zero_pct = (qty_zero / len(df)) * 100

print(f"   - Quantités négatives: {qty_negative:,} ({qty_negative_pct:.2f}%)")
print(f"   - Quantités nulles: {qty_zero:,} ({qty_zero_pct:.2f}%)")

if qty_negative > 0:
    print(f"\n   Distribution des quantités négatives:")
    print(df[df['Quantity'] < 0]['Quantity'].describe())
    print(f"\n   Exemple de quantités négatives:")
    display(df[df['Quantity'] < 0][['Invoice', 'StockCode', 'Description', 'Quantity', 'Price']].head())

# 3. Prix négatifs ou nuls
print(f"\n3. PRIX NÉGATIFS OU NULS:")
price_negative = (df['Price'] < 0).sum()
price_negative_pct = (price_negative / len(df)) * 100
price_zero = (df['Price'] == 0).sum()
price_zero_pct = (price_zero / len(df)) * 100

print(f"   - Prix négatifs: {price_negative:,} ({price_negative_pct:.2f}%)")
print(f"   - Prix nuls: {price_zero:,} ({price_zero_pct:.2f}%)")

if price_negative > 0:
    print(f"\n   Exemple de prix négatifs:")
    display(df[df['Price'] < 0][['Invoice', 'StockCode', 'Description', 'Quantity', 'Price']].head())

if price_zero > 0:
    print(f"\n   Exemple de prix nuls (peut-être des échantillons gratuits ou erreurs):")
    display(df[df['Price'] == 0][['Invoice', 'StockCode', 'Description', 'Quantity', 'Price']].head())

# 4. Customer ID manquants - Impact détaillé
print(f"\n4. CUSTOMER ID MANQUANTS - ANALYSE DÉTAILLÉE:")
missing_customer = df['Customer ID'].isnull().sum()
missing_customer_pct = (missing_customer / len(df)) * 100

print(f"   - Lignes sans Customer ID: {missing_customer:,} ({missing_customer_pct:.2f}%)")
print(f"   - Impact: Ces transactions ne peuvent pas être utilisées pour:")
print(f"     • Analyse de cohortes")
print(f"     • Segmentation RFM")
print(f"     • Calcul de CLV")
print(f"     • Analyse de rétention")

if missing_customer > 0:
    # Comparer les caractéristiques avec/sans Customer ID
    with_customer = df[df['Customer ID'].notna()]
    without_customer = df[df['Customer ID'].isna()]
    
    print(f"\n   Comparaison avec/sans Customer ID:")
    print(f"   Avec Customer ID:")
    print(f"     - Nombre de transactions: {len(with_customer):,}")
    if 'TotalAmount' not in df.columns:
        df['TotalAmount'] = df['Quantity'] * df['Price']
    print(f"     - Montant total: £{with_customer['TotalAmount'].sum():,.2f}")
    print(f"     - Panier moyen: £{with_customer['TotalAmount'].mean():.2f}")
    
    print(f"\n   Sans Customer ID:")
    print(f"     - Nombre de transactions: {len(without_customer):,}")
    print(f"     - Montant total: £{without_customer['TotalAmount'].sum():,.2f}")
    print(f"     - Panier moyen: £{without_customer['TotalAmount'].mean():.2f}")

# 5. StockCode invalides ou spéciaux
print(f"\n5. STOCKCODE SPÉCIAUX:")
stockcode_special = df[df['StockCode'].str.contains('POST|BANK|FEE|ADJUST', case=False, na=False)]
print(f"   - Codes spéciaux (POST, BANK, FEE, ADJUST): {len(stockcode_special):,}")
if len(stockcode_special) > 0:
    print(f"   Exemples:")
    display(stockcode_special[['Invoice', 'StockCode', 'Description', 'Quantity', 'Price']].head())

# 6. Dates incohérentes
print(f"\n6. DATES:")
print(f"   - Date minimale: {df['InvoiceDate'].min()}")
print(f"   - Date maximale: {df['InvoiceDate'].max()}")
print(f"   - Période couverte: {(df['InvoiceDate'].max() - df['InvoiceDate'].min()).days} jours")

# Vérifier s'il y a des dates futures (par rapport à une date de référence)
# Assumer que le dataset est de 2010-2011
max_expected_date = pd.Timestamp('2012-12-31')
future_dates = df[df['InvoiceDate'] > max_expected_date]
if len(future_dates) > 0:
    print(f"   ⚠️ ATTENTION: {len(future_dates):,} transactions avec dates > {max_expected_date}")

print("\n" + "="*80)

In [ ]:
# Visualisation des incohérences
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Distribution des catégories d'anomalies
anomaly_counts = pd.Series({
    'Annulations': cancellations,
    'Qty négative': qty_negative,
    'Qty nulle': qty_zero,
    'Prix négatif': price_negative,
    'Prix nul': price_zero,
    'Customer ID manquant': missing_customer
})
colors = ['#E74C3C', '#E67E22', '#F39C12', '#3498DB', '#9B59B6', '#1ABC9C']
axes[0, 0].barh(anomaly_counts.index, anomaly_counts.values, color=colors)
axes[0, 0].set_xlabel('Nombre de lignes', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Nombre de lignes par type d\'anomalie', fontsize=14, fontweight='bold')
axes[0, 0].grid(axis='x', alpha=0.3)
for i, v in enumerate(anomaly_counts.values):
    axes[0, 0].text(v, i, f' {v:,}', va='center', fontsize=10)

# 2. Pourcentage d'anomalies
anomaly_pcts = (anomaly_counts / len(df)) * 100
axes[0, 1].barh(anomaly_pcts.index, anomaly_pcts.values, color=colors)
axes[0, 1].set_xlabel('Pourcentage (%)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Pourcentage de lignes par type d\'anomalie', fontsize=14, fontweight='bold')
axes[0, 1].grid(axis='x', alpha=0.3)
for i, v in enumerate(anomaly_pcts.values):
    axes[0, 1].text(v, i, f' {v:.2f}%', va='center', fontsize=10)

# 3. Distribution des quantités (focus sur les valeurs extrêmes)
axes[1, 0].hist(df['Quantity'], bins=100, color='#3498DB', alpha=0.7, edgecolor='black')
axes[1, 0].set_xlabel('Quantité', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Fréquence', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Distribution des quantités (toutes valeurs)', fontsize=14, fontweight='bold')
axes[1, 0].axvline(0, color='red', linestyle='--', linewidth=2, label='Zéro')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# 4. Distribution des prix (focus sur les valeurs extrêmes)
axes[1, 1].hist(df[df['Price'] <= df['Price'].quantile(0.95)]['Price'], bins=100, color='#2ECC71', alpha=0.7, edgecolor='black')
axes[1, 1].set_xlabel('Prix (£)', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Fréquence', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Distribution des prix (95e percentile)', fontsize=14, fontweight='bold')
axes[1, 1].axvline(0, color='red', linestyle='--', linewidth=2, label='Zéro')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("DÉCISIONS DE NETTOYAGE RECOMMANDÉES".center(80))
print("="*80)
print("""
1. ANNULATIONS (Invoice avec 'C'):
   ➤ Décision: TRAITER SÉPARÉMENT
   ➤ Action: Créer un flag 'IsCancellation' et analyser à part
   ➤ Pour l'analyse client: Exclure ou matcher avec les ventes originales

2. QUANTITÉS NÉGATIVES:
   ➤ Décision: EXCLURE pour l'analyse principale
   ➤ Raison: Représentent des retours/annulations
   ➤ Action: Filtrer avec df[df['Quantity'] > 0]

3. QUANTITÉS NULLES:
   ➤ Décision: EXCLURE
   ➤ Raison: Pas de valeur business
   ➤ Action: Inclure dans le filtre df[df['Quantity'] > 0]

4. PRIX NÉGATIFS:
   ➤ Décision: EXCLURE
   ➤ Raison: Incohérent pour une vente
   ➤ Action: Filtrer avec df[df['Price'] > 0]

5. PRIX NULS:
   ➤ Décision: EXCLURE
   ➤ Raison: Pas de contribution au revenu
   ➤ Action: Inclure dans le filtre df[df['Price'] > 0]

6. CUSTOMER ID MANQUANTS:
   ➤ Décision: EXCLURE pour analyses client (cohortes, RFM, CLV)
   ➤ Raison: Impossible de tracker le comportement client
   ➤ Action: Filtrer avec df[df['Customer ID'].notna()]
   ➤ Note: Conserver pour analyse produit/globale si nécessaire
""")

### 4.4 Analyse des Outliers (Valeurs Extrêmes)

Identification et analyse des valeurs aberrantes dans les variables numériques.

In [ ]:
print("="*80)
print("ANALYSE DES OUTLIERS (VALEURS EXTRÊMES)".center(80))
print("="*80)

# Travailler sur un subset sans les valeurs négatives pour l'analyse des outliers
df_positive = df[(df['Quantity'] > 0) & (df['Price'] > 0)].copy()

print(f"\nDataset pour analyse des outliers:")
print(f"  - Lignes avec Quantity > 0 et Price > 0: {len(df_positive):,}")
print(f"  - Lignes exclues: {len(df) - len(df_positive):,}")

# 1. ANALYSE DES QUANTITÉS
print("\n" + "="*80)
print("1. OUTLIERS - QUANTITY".center(80))
print("="*80)

# Statistiques descriptives
qty_stats = df_positive['Quantity'].describe()
print("\nStatistiques descriptives:")
print(qty_stats)

# Calcul IQR
Q1_qty = df_positive['Quantity'].quantile(0.25)
Q3_qty = df_positive['Quantity'].quantile(0.75)
IQR_qty = Q3_qty - Q1_qty
lower_bound_qty = Q1_qty - 1.5 * IQR_qty
upper_bound_qty = Q3_qty + 1.5 * IQR_qty

print(f"\nMéthode IQR:")
print(f"  - Q1 (25e percentile): {Q1_qty:.2f}")
print(f"  - Q3 (75e percentile): {Q3_qty:.2f}")
print(f"  - IQR: {IQR_qty:.2f}")
print(f"  - Seuil inférieur (Q1 - 1.5*IQR): {lower_bound_qty:.2f}")
print(f"  - Seuil supérieur (Q3 + 1.5*IQR): {upper_bound_qty:.2f}")

outliers_qty = df_positive[(df_positive['Quantity'] < lower_bound_qty) | (df_positive['Quantity'] > upper_bound_qty)]
print(f"\n  - Nombre d'outliers: {len(outliers_qty):,} ({(len(outliers_qty)/len(df_positive))*100:.2f}%)")
print(f"  - Valeur maximale: {df_positive['Quantity'].max():,.0f}")

# Percentiles
print("\nPercentiles de Quantity:")
percentiles = [50, 75, 90, 95, 99, 99.9]
for p in percentiles:
    val = df_positive['Quantity'].quantile(p/100)
    print(f"  - {p}e percentile: {val:.2f}")

# 2. ANALYSE DES PRIX
print("\n" + "="*80)
print("2. OUTLIERS - PRICE".center(80))
print("="*80)

# Statistiques descriptives
price_stats = df_positive['Price'].describe()
print("\nStatistiques descriptives:")
print(price_stats)

# Calcul IQR
Q1_price = df_positive['Price'].quantile(0.25)
Q3_price = df_positive['Price'].quantile(0.75)
IQR_price = Q3_price - Q1_price
lower_bound_price = Q1_price - 1.5 * IQR_price
upper_bound_price = Q3_price + 1.5 * IQR_price

print(f"\nMéthode IQR:")
print(f"  - Q1 (25e percentile): £{Q1_price:.2f}")
print(f"  - Q3 (75e percentile): £{Q3_price:.2f}")
print(f"  - IQR: £{IQR_price:.2f}")
print(f"  - Seuil inférieur (Q1 - 1.5*IQR): £{lower_bound_price:.2f}")
print(f"  - Seuil supérieur (Q3 + 1.5*IQR): £{upper_bound_price:.2f}")

outliers_price = df_positive[(df_positive['Price'] < lower_bound_price) | (df_positive['Price'] > upper_bound_price)]
print(f"\n  - Nombre d'outliers: {len(outliers_price):,} ({(len(outliers_price)/len(df_positive))*100:.2f}%)")
print(f"  - Valeur maximale: £{df_positive['Price'].max():,.2f}")

# Percentiles
print("\nPercentiles de Price:")
for p in percentiles:
    val = df_positive['Price'].quantile(p/100)
    print(f"  - {p}e percentile: £{val:.2f}")

# 3. ANALYSE DU MONTANT TOTAL
print("\n" + "="*80)
print("3. OUTLIERS - TOTAL AMOUNT (Quantity × Price)".center(80))
print("="*80)

if 'TotalAmount' not in df_positive.columns:
    df_positive['TotalAmount'] = df_positive['Quantity'] * df_positive['Price']

# Statistiques descriptives
amount_stats = df_positive['TotalAmount'].describe()
print("\nStatistiques descriptives:")
print(amount_stats)

# Calcul IQR
Q1_amount = df_positive['TotalAmount'].quantile(0.25)
Q3_amount = df_positive['TotalAmount'].quantile(0.75)
IQR_amount = Q3_amount - Q1_amount
lower_bound_amount = Q1_amount - 1.5 * IQR_amount
upper_bound_amount = Q3_amount + 1.5 * IQR_amount

print(f"\nMéthode IQR:")
print(f"  - Q1 (25e percentile): £{Q1_amount:.2f}")
print(f"  - Q3 (75e percentile): £{Q3_amount:.2f}")
print(f"  - IQR: £{IQR_amount:.2f}")
print(f"  - Seuil inférieur (Q1 - 1.5*IQR): £{lower_bound_amount:.2f}")
print(f"  - Seuil supérieur (Q3 + 1.5*IQR): £{upper_bound_amount:.2f}")

outliers_amount = df_positive[(df_positive['TotalAmount'] < lower_bound_amount) | (df_positive['TotalAmount'] > upper_bound_amount)]
print(f"\n  - Nombre d'outliers: {len(outliers_amount):,} ({(len(outliers_amount)/len(df_positive))*100:.2f}%)")
print(f"  - Valeur maximale: £{df_positive['TotalAmount'].max():,.2f}")

# Percentiles
print("\nPercentiles de TotalAmount:")
for p in percentiles:
    val = df_positive['TotalAmount'].quantile(p/100)
    print(f"  - {p}e percentile: £{val:.2f}")

print("\n" + "="*80)

In [ ]:
# Visualisation des outliers avec box plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Box plot Quantity (toutes valeurs)
axes[0, 0].boxplot(df_positive['Quantity'], vert=True)
axes[0, 0].set_ylabel('Quantité', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Box Plot - Quantity (toutes valeurs)', fontsize=14, fontweight='bold')
axes[0, 0].grid(alpha=0.3)

# 2. Box plot Quantity (sans outliers extrêmes, 99e percentile)
qty_99 = df_positive['Quantity'].quantile(0.99)
axes[0, 1].boxplot(df_positive[df_positive['Quantity'] <= qty_99]['Quantity'], vert=True)
axes[0, 1].set_ylabel('Quantité', fontsize=12, fontweight='bold')
axes[0, 1].set_title(f'Box Plot - Quantity (≤ 99e percentile = {qty_99:.0f})', fontsize=14, fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# 3. Histogramme Quantity (log scale)
axes[0, 2].hist(df_positive['Quantity'], bins=100, color='#3498DB', alpha=0.7, edgecolor='black')
axes[0, 2].set_xlabel('Quantité', fontsize=12, fontweight='bold')
axes[0, 2].set_ylabel('Fréquence (échelle log)', fontsize=12, fontweight='bold')
axes[0, 2].set_title('Distribution - Quantity', fontsize=14, fontweight='bold')
axes[0, 2].set_yscale('log')
axes[0, 2].axvline(upper_bound_qty, color='red', linestyle='--', linewidth=2, label=f'Seuil IQR = {upper_bound_qty:.0f}')
axes[0, 2].legend()
axes[0, 2].grid(alpha=0.3)

# 4. Box plot Price (toutes valeurs)
axes[1, 0].boxplot(df_positive['Price'], vert=True)
axes[1, 0].set_ylabel('Prix (£)', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Box Plot - Price (toutes valeurs)', fontsize=14, fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# 5. Box plot Price (sans outliers extrêmes, 99e percentile)
price_99 = df_positive['Price'].quantile(0.99)
axes[1, 1].boxplot(df_positive[df_positive['Price'] <= price_99]['Price'], vert=True)
axes[1, 1].set_ylabel('Prix (£)', fontsize=12, fontweight='bold')
axes[1, 1].set_title(f'Box Plot - Price (≤ 99e percentile = £{price_99:.2f})', fontsize=14, fontweight='bold')
axes[1, 1].grid(alpha=0.3)

# 6. Histogramme Price (log scale)
axes[1, 2].hist(df_positive['Price'], bins=100, color='#2ECC71', alpha=0.7, edgecolor='black')
axes[1, 2].set_xlabel('Prix (£)', fontsize=12, fontweight='bold')
axes[1, 2].set_ylabel('Fréquence (échelle log)', fontsize=12, fontweight='bold')
axes[1, 2].set_title('Distribution - Price', fontsize=14, fontweight='bold')
axes[1, 2].set_yscale('log')
axes[1, 2].axvline(upper_bound_price, color='red', linestyle='--', linewidth=2, label=f'Seuil IQR = £{upper_bound_price:.2f}')
axes[1, 2].legend()
axes[1, 2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Visualisation TotalAmount
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Box plot TotalAmount (toutes valeurs)
axes[0].boxplot(df_positive['TotalAmount'], vert=True)
axes[0].set_ylabel('Montant Total (£)', fontsize=12, fontweight='bold')
axes[0].set_title('Box Plot - TotalAmount (toutes valeurs)', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)

# Box plot TotalAmount (99e percentile)
amount_99 = df_positive['TotalAmount'].quantile(0.99)
axes[1].boxplot(df_positive[df_positive['TotalAmount'] <= amount_99]['TotalAmount'], vert=True)
axes[1].set_ylabel('Montant Total (£)', fontsize=12, fontweight='bold')
axes[1].set_title(f'Box Plot - TotalAmount (≤ 99e percentile = £{amount_99:.2f})', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)

# Histogramme TotalAmount (log scale)
axes[2].hist(df_positive['TotalAmount'], bins=100, color='#E67E22', alpha=0.7, edgecolor='black')
axes[2].set_xlabel('Montant Total (£)', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Fréquence (échelle log)', fontsize=12, fontweight='bold')
axes[2].set_title('Distribution - TotalAmount', fontsize=14, fontweight='bold')
axes[2].set_yscale('log')
axes[2].axvline(upper_bound_amount, color='red', linestyle='--', linewidth=2, label=f'Seuil IQR = £{upper_bound_amount:.2f}')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("DÉCISION SUR LE TRAITEMENT DES OUTLIERS".center(80))
print("="*80)
print("""
APPROCHE RECOMMANDÉE: CONSERVER LES OUTLIERS

Raisons:
1. Les outliers peuvent représenter des commandes en gros (B2B) légitimes
2. Supprimer les outliers pourrait biaiser l'analyse du revenu total
3. Les valeurs extrêmes contribuent significativement au CA

Actions à prendre:
➤ CONSERVER tous les outliers pour l'analyse globale du revenu
➤ DOCUMENTER la présence d'outliers dans les rapports
➤ CRÉER des analyses séparées:
  - Analyse incluant tous les clients (vue complète du business)
  - Analyse excluant le top 1% (comportement client typique)
➤ SEGMENTER les clients par taille de commande:
  - Petits acheteurs (< 90e percentile)
  - Acheteurs moyens (90e-99e percentile)
  - Gros acheteurs (> 99e percentile) → Potentiellement B2B

Alternative (si nécessaire):
➤ Pour certaines analyses spécifiques (ex: panier moyen du particulier),
  filtrer sur des seuils raisonnables (ex: ≤ 99e percentile)

Note importante:
➤ Les outliers dans un contexte retail sont souvent INFORMATIFS et non ABERRANTS
➤ Ils révèlent des segments de clients à haute valeur (VIP, B2B)
➤ Leur conservation est essentielle pour une analyse CLV précise
""")

### 4.5 Granularité et Continuité Temporelle

Analyse de la distribution des transactions dans le temps pour identifier les patterns, gaps et périodes manquantes.

In [ ]:
print("="*80)
print("ANALYSE DE LA GRANULARITÉ TEMPORELLE".center(80))
print("="*80)

# Travailler avec les données positives et complètes
df_temporal = df[(df['Quantity'] > 0) & (df['Price'] > 0) & (df['Customer ID'].notna())].copy()
if 'TotalAmount' not in df_temporal.columns:
    df_temporal['TotalAmount'] = df_temporal['Quantity'] * df_temporal['Price']

print(f"\nDataset pour analyse temporelle:")
print(f"  - Nombre de transactions: {len(df_temporal):,}")
print(f"  - Période: {df_temporal['InvoiceDate'].min()} à {df_temporal['InvoiceDate'].max()}")
print(f"  - Durée: {(df_temporal['InvoiceDate'].max() - df_temporal['InvoiceDate'].min()).days} jours")

# Extraction des composantes temporelles
df_temporal['Year'] = df_temporal['InvoiceDate'].dt.year
df_temporal['Month'] = df_temporal['InvoiceDate'].dt.month
df_temporal['Day'] = df_temporal['InvoiceDate'].dt.day
df_temporal['DayOfWeek'] = df_temporal['InvoiceDate'].dt.dayofweek  # 0=Lundi, 6=Dimanche
df_temporal['DayName'] = df_temporal['InvoiceDate'].dt.day_name()
df_temporal['Hour'] = df_temporal['InvoiceDate'].dt.hour
df_temporal['Date'] = df_temporal['InvoiceDate'].dt.date
df_temporal['YearMonth'] = df_temporal['InvoiceDate'].dt.to_period('M')

# 1. DISTRIBUTION PAR ANNÉE
print("\n" + "="*80)
print("1. DISTRIBUTION PAR ANNÉE".center(80))
print("="*80)

year_stats = df_temporal.groupby('Year').agg({
    'Invoice': 'count',
    'TotalAmount': 'sum',
    'Customer ID': 'nunique'
}).rename(columns={
    'Invoice': 'NbTransactions',
    'TotalAmount': 'Revenu',
    'Customer ID': 'NbClients'
})

print("\nStatistiques par année:")
display(year_stats.style.format({
    'NbTransactions': '{:,.0f}',
    'Revenu': '£{:,.2f}',
    'NbClients': '{:,.0f}'
}))

# 2. DISTRIBUTION PAR MOIS
print("\n" + "="*80)
print("2. DISTRIBUTION PAR MOIS (ANNÉE-MOIS)".center(80))
print("="*80)

month_stats = df_temporal.groupby('YearMonth').agg({
    'Invoice': 'count',
    'TotalAmount': 'sum',
    'Customer ID': 'nunique'
}).rename(columns={
    'Invoice': 'NbTransactions',
    'TotalAmount': 'Revenu',
    'Customer ID': 'NbClients'
})

print(f"\nNombre de mois dans le dataset: {len(month_stats)}")
print(f"Mois avec le plus de transactions: {month_stats['NbTransactions'].idxmax()} ({month_stats['NbTransactions'].max():,} transactions)")
print(f"Mois avec le moins de transactions: {month_stats['NbTransactions'].idxmin()} ({month_stats['NbTransactions'].min():,} transactions)")
print(f"\nAperçu des 10 premiers mois:")
display(month_stats.head(10).style.format({
    'NbTransactions': '{:,.0f}',
    'Revenu': '£{:,.2f}',
    'NbClients': '{:,.0f}'
}))

# 3. DISTRIBUTION PAR JOUR DE LA SEMAINE
print("\n" + "="*80)
print("3. DISTRIBUTION PAR JOUR DE LA SEMAINE".center(80))
print("="*80)

day_stats = df_temporal.groupby('DayName').agg({
    'Invoice': 'count',
    'TotalAmount': 'sum'
}).rename(columns={
    'Invoice': 'NbTransactions',
    'TotalAmount': 'Revenu'
})

# Réordonner les jours de la semaine
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_stats = day_stats.reindex(day_order)

print("\nTransactions par jour de la semaine:")
display(day_stats.style.format({
    'NbTransactions': '{:,.0f}',
    'Revenu': '£{:,.2f}'
}))

# 4. DISTRIBUTION PAR HEURE DE LA JOURNÉE
print("\n" + "="*80)
print("4. DISTRIBUTION PAR HEURE DE LA JOURNÉE".center(80))
print("="*80)

hour_stats = df_temporal.groupby('Hour').agg({
    'Invoice': 'count',
    'TotalAmount': 'sum'
}).rename(columns={
    'Invoice': 'NbTransactions',
    'TotalAmount': 'Revenu'
})

print(f"\nHeure la plus active: {hour_stats['NbTransactions'].idxmax()}h ({hour_stats['NbTransactions'].max():,} transactions)")
print(f"Heure la moins active: {hour_stats['NbTransactions'].idxmin()}h ({hour_stats['NbTransactions'].min():,} transactions)")
print("\nTop 5 heures les plus actives:")
display(hour_stats.nlargest(5, 'NbTransactions').style.format({
    'NbTransactions': '{:,.0f}',
    'Revenu': '£{:,.2f}'
}))

# 5. ANALYSE DES GAPS TEMPORELS
print("\n" + "="*80)
print("5. ANALYSE DES GAPS TEMPORELS (PÉRIODES SANS TRANSACTIONS)".center(80))
print("="*80)

# Compter les transactions par jour
daily_counts = df_temporal.groupby('Date').size().reset_index(name='NbTransactions')
daily_counts['Date'] = pd.to_datetime(daily_counts['Date'])

# Créer une série temporelle complète
date_range = pd.date_range(start=df_temporal['InvoiceDate'].min().date(), 
                           end=df_temporal['InvoiceDate'].max().date(), 
                           freq='D')
full_dates = pd.DataFrame({'Date': date_range})
daily_complete = full_dates.merge(daily_counts, on='Date', how='left').fillna(0)

# Identifier les jours sans transactions
days_without_transactions = daily_complete[daily_complete['NbTransactions'] == 0]

print(f"\nJours totaux dans la période: {len(daily_complete)}")
print(f"Jours avec transactions: {len(daily_complete[daily_complete['NbTransactions'] > 0])}")
print(f"Jours sans transactions: {len(days_without_transactions)}")
print(f"Pourcentage de jours avec activité: {(len(daily_complete[daily_complete['NbTransactions'] > 0])/len(daily_complete))*100:.2f}%")

if len(days_without_transactions) > 0:
    print(f"\nExemples de jours sans transactions (peut-être weekends ou jours fériés):")
    print(days_without_transactions['Date'].head(10).to_list())

# 6. FRÉQUENCE MOYENNE DES TRANSACTIONS
print("\n" + "="*80)
print("6. FRÉQUENCE MOYENNE DES TRANSACTIONS".center(80))
print("="*80)

total_transactions = len(df_temporal)
total_days = (df_temporal['InvoiceDate'].max() - df_temporal['InvoiceDate'].min()).days
total_hours = total_days * 24

print(f"\nFréquence des transactions:")
print(f"  - Transactions par jour (moyenne): {total_transactions/total_days:.2f}")
print(f"  - Transactions par heure (moyenne): {total_transactions/total_hours:.2f}")
print(f"  - Transactions par minute (moyenne): {total_transactions/(total_hours*60):.2f}")

# Statistiques des transactions quotidiennes
daily_transaction_counts = daily_complete[daily_complete['NbTransactions'] > 0]['NbTransactions']
print(f"\nStatistiques des jours avec activité:")
print(f"  - Minimum transactions/jour: {daily_transaction_counts.min():.0f}")
print(f"  - Maximum transactions/jour: {daily_transaction_counts.max():.0f}")
print(f"  - Moyenne transactions/jour: {daily_transaction_counts.mean():.2f}")
print(f"  - Médiane transactions/jour: {daily_transaction_counts.median():.0f}")

print("\n" + "="*80)

In [ ]:
# Visualisations temporelles
fig, axes = plt.subplots(3, 2, figsize=(18, 16))

# 1. Évolution mensuelle du nombre de transactions
month_stats_plot = month_stats.reset_index()
month_stats_plot['YearMonth_str'] = month_stats_plot['YearMonth'].astype(str)
axes[0, 0].plot(range(len(month_stats_plot)), month_stats_plot['NbTransactions'], marker='o', linewidth=2, markersize=6, color='#3498DB')
axes[0, 0].set_xlabel('Mois', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Nombre de transactions', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Évolution mensuelle du nombre de transactions', fontsize=14, fontweight='bold')
axes[0, 0].grid(alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Évolution mensuelle du revenu
axes[0, 1].plot(range(len(month_stats_plot)), month_stats_plot['Revenu'], marker='o', linewidth=2, markersize=6, color='#2ECC71')
axes[0, 1].set_xlabel('Mois', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Revenu (£)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Évolution mensuelle du revenu', fontsize=14, fontweight='bold')
axes[0, 1].grid(alpha=0.3)
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Transactions par jour de la semaine
day_stats_plot = day_stats.reset_index()
colors_days = ['#3498DB', '#3498DB', '#3498DB', '#3498DB', '#3498DB', '#E74C3C', '#E74C3C']
axes[1, 0].bar(day_stats_plot['DayName'], day_stats_plot['NbTransactions'], color=colors_days, edgecolor='black')
axes[1, 0].set_xlabel('Jour de la semaine', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Nombre de transactions', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Transactions par jour de la semaine', fontsize=14, fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)
axes[1, 0].tick_params(axis='x', rotation=45)
for i, v in enumerate(day_stats_plot['NbTransactions']):
    axes[1, 0].text(i, v, f'{v:,.0f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# 4. Revenu par jour de la semaine
axes[1, 1].bar(day_stats_plot['DayName'], day_stats_plot['Revenu'], color=colors_days, edgecolor='black')
axes[1, 1].set_xlabel('Jour de la semaine', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Revenu (£)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Revenu par jour de la semaine', fontsize=14, fontweight='bold')
axes[1, 1].grid(axis='y', alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)

# 5. Transactions par heure de la journée
hour_stats_plot = hour_stats.reset_index()
axes[2, 0].bar(hour_stats_plot['Hour'], hour_stats_plot['NbTransactions'], color='#9B59B6', edgecolor='black')
axes[2, 0].set_xlabel('Heure de la journée', fontsize=12, fontweight='bold')
axes[2, 0].set_ylabel('Nombre de transactions', fontsize=12, fontweight='bold')
axes[2, 0].set_title('Transactions par heure de la journée', fontsize=14, fontweight='bold')
axes[2, 0].grid(axis='y', alpha=0.3)
axes[2, 0].set_xticks(range(0, 24, 2))

# 6. Transactions quotidiennes (série temporelle)
daily_complete_plot = daily_complete.copy()
axes[2, 1].plot(daily_complete_plot['Date'], daily_complete_plot['NbTransactions'], linewidth=1, color='#E67E22', alpha=0.7)
axes[2, 1].set_xlabel('Date', fontsize=12, fontweight='bold')
axes[2, 1].set_ylabel('Nombre de transactions', fontsize=12, fontweight='bold')
axes[2, 1].set_title('Série temporelle des transactions quotidiennes', fontsize=14, fontweight='bold')
axes[2, 1].grid(alpha=0.3)
axes[2, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("INTERPRÉTATIONS ET INSIGHTS TEMPORELS".center(80))
print("="*80)
print("""
📊 OBSERVATIONS CLÉS:

1. TENDANCE MENSUELLE:
   ➤ Observer si la tendance est croissante, stable ou décroissante
   ➤ Identifier les pics saisonniers (ex: fin d'année pour les fêtes)
   ➤ Détecter les anomalies ou baisses inhabituelles

2. JOURS DE LA SEMAINE:
   ➤ Les weekends (Saturday, Sunday) montrent généralement moins d'activité
   ➤ Les jours ouvrables (Monday-Friday) ont plus de transactions
   ➤ Impact business: Adapter les ressources selon le jour

3. HEURES DE LA JOURNÉE:
   ➤ Les heures de bureau (9h-17h) sont les plus actives
   ➤ Peu ou pas d'activité la nuit
   ➤ Impact business: Planifier les opérations et le support client

4. GAPS TEMPORELS:
   ➤ Jours sans transactions = weekends, jours fériés, ou problèmes techniques
   ➤ Important pour l'analyse de cohortes (ne pas compter ces jours comme inactifs)

5. SAISONNALITÉ:
   ➤ Identifier les mois/périodes à forte demande
   ➤ Planifier les stocks et campagnes marketing en conséquence
   ➤ Analyser les cohortes en tenant compte de la saisonnalité

DÉCISIONS POUR LA SUITE:
✓ Utiliser ces patterns pour l'analyse de rétention (exclure weekends/jours fériés)
✓ Créer des segments temporels pour l'analyse de cohortes
✓ Tenir compte de la saisonnalité dans les prévisions de CLV
✓ Adapter les stratégies marketing selon les périodes de forte/faible activité
""")

---

## Synthèse de l'Analyse de Qualité (Section 4)

### Récapitulatif des Problèmes Identifiés

| Catégorie | Problème | Nombre | Impact | Décision |
|-----------|----------|--------|--------|----------|
| **Valeurs manquantes** | Customer ID manquant | Variable | CRITIQUE | Exclure pour analyse client |
| | Description manquante | Variable | MODÉRÉ | Marquer comme 'Inconnu' |
| **Doublons** | Doublons complets | Variable | FAIBLE | Supprimer si présents |
| | Doublons partiels | Variable | À évaluer | Analyser cas par cas |
| **Règles métier** | Factures annulation (C) | Variable | ÉLEVÉ | Traiter séparément |
| | Quantités négatives/nulles | Variable | ÉLEVÉ | Exclure |
| | Prix négatifs/nuls | Variable | ÉLEVÉ | Exclure |
| **Outliers** | Quantités extrêmes | Variable | INFORMATIF | Conserver |
| | Prix extrêmes | Variable | INFORMATIF | Conserver |
| **Temporalité** | Gaps temporels | Variable | NORMAL | Weekends/fériés attendus |

### Pipeline de Nettoyage Recommandé

```python
# Étape 1: Supprimer les doublons complets
df_clean = df.drop_duplicates()

# Étape 2: Exclure les annulations
df_clean = df_clean[~df_clean['Invoice'].astype(str).str.startswith('C')]

# Étape 3: Filtrer les valeurs invalides
df_clean = df_clean[
    (df_clean['Quantity'] > 0) &
    (df_clean['Price'] > 0) &
    (df_clean['Customer ID'].notna())
]

# Étape 4: Créer les variables calculées
df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['Price']

# Étape 5: Variables temporelles
df_clean['InvoiceMonth'] = df_clean['InvoiceDate'].dt.to_period('M')
df_clean['Year'] = df_clean['InvoiceDate'].dt.year
df_clean['Month'] = df_clean['InvoiceDate'].dt.month
df_clean['DayOfWeek'] = df_clean['InvoiceDate'].dt.dayofweek
df_clean['Hour'] = df_clean['InvoiceDate'].dt.hour
```

### Prochaines Étapes (Sections 5+)

Les sections suivantes du notebook pourront maintenant se concentrer sur :

1. **Visualisations exploratoires** (Section 5) : Patterns de vente, géographie, produits
2. **Analyse de cohortes** : Identification et suivi des cohortes d'acquisition
3. **Segmentation RFM** : Calcul des scores et création des segments
4. **Calcul de CLV** : Valorisation des clients et prévisions

---

---

## 5. Visualisations Exploratoires

Cette section présente les visualisations clés pour comprendre les patterns de vente, le comportement client et identifier des opportunités business.

### 5.1 Graphique 1 : Distributions des Variables Clés

Analyse des distributions de Quantity, Price et TotalAmount pour comprendre la structure des transactions.

In [ ]:
# Préparer les données pour l'analyse (uniquement transactions valides)
df_analysis = df[
    (df['Quantity'] > 0) & 
    (df['Price'] > 0) & 
    (df['Customer ID'].notna()) &
    (~df['Invoice'].astype(str).str.startswith('C'))
].copy()

# Créer TotalAmount si pas déjà fait
df_analysis['TotalAmount'] = df_analysis['Quantity'] * df_analysis['Price']

print(f"Dataset pour visualisations : {len(df_analysis):,} transactions valides")
print(f"Période : {df_analysis['InvoiceDate'].min()} à {df_analysis['InvoiceDate'].max()}")

# GRAPHIQUE 1: Distributions (Quantity, Price, TotalAmount)
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Distribution des quantités (avec KDE)
axes[0].hist(df_analysis['Quantity'], bins=100, color='#3498DB', alpha=0.7, edgecolor='black', density=True)
# Ajouter KDE
from scipy import stats
kde_qty = stats.gaussian_kde(df_analysis['Quantity'])
x_qty = np.linspace(df_analysis['Quantity'].min(), df_analysis['Quantity'].quantile(0.99), 1000)
axes[0].plot(x_qty, kde_qty(x_qty), 'r-', linewidth=2, label='KDE')
axes[0].set_xlabel('Quantité', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Densité', fontsize=13, fontweight='bold')
axes[0].set_title('Distribution des Quantités', fontsize=15, fontweight='bold')
axes[0].axvline(df_analysis['Quantity'].median(), color='green', linestyle='--', linewidth=2, label=f'Médiane = {df_analysis["Quantity"].median():.0f}')
axes[0].axvline(df_analysis['Quantity'].mean(), color='orange', linestyle='--', linewidth=2, label=f'Moyenne = {df_analysis["Quantity"].mean():.1f}')
axes[0].legend()
axes[0].grid(alpha=0.3)
axes[0].set_xlim(0, df_analysis['Quantity'].quantile(0.99))

# Distribution des prix (avec KDE)
axes[1].hist(df_analysis['Price'], bins=100, color='#2ECC71', alpha=0.7, edgecolor='black', density=True)
kde_price = stats.gaussian_kde(df_analysis['Price'])
x_price = np.linspace(df_analysis['Price'].min(), df_analysis['Price'].quantile(0.99), 1000)
axes[1].plot(x_price, kde_price(x_price), 'r-', linewidth=2, label='KDE')
axes[1].set_xlabel('Prix Unitaire (£)', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Densité', fontsize=13, fontweight='bold')
axes[1].set_title('Distribution des Prix Unitaires', fontsize=15, fontweight='bold')
axes[1].axvline(df_analysis['Price'].median(), color='green', linestyle='--', linewidth=2, label=f'Médiane = £{df_analysis["Price"].median():.2f}')
axes[1].axvline(df_analysis['Price'].mean(), color='orange', linestyle='--', linewidth=2, label=f'Moyenne = £{df_analysis["Price"].mean():.2f}')
axes[1].legend()
axes[1].grid(alpha=0.3)
axes[1].set_xlim(0, df_analysis['Price'].quantile(0.99))

# Distribution du montant total (avec KDE)
axes[2].hist(df_analysis['TotalAmount'], bins=100, color='#E67E22', alpha=0.7, edgecolor='black', density=True)
kde_amount = stats.gaussian_kde(df_analysis['TotalAmount'])
x_amount = np.linspace(df_analysis['TotalAmount'].min(), df_analysis['TotalAmount'].quantile(0.99), 1000)
axes[2].plot(x_amount, kde_amount(x_amount), 'r-', linewidth=2, label='KDE')
axes[2].set_xlabel('Montant Total (£)', fontsize=13, fontweight='bold')
axes[2].set_ylabel('Densité', fontsize=13, fontweight='bold')
axes[2].set_title('Distribution du Montant Total par Transaction', fontsize=15, fontweight='bold')
axes[2].axvline(df_analysis['TotalAmount'].median(), color='green', linestyle='--', linewidth=2, label=f'Médiane = £{df_analysis["TotalAmount"].median():.2f}')
axes[2].axvline(df_analysis['TotalAmount'].mean(), color='orange', linestyle='--', linewidth=2, label=f'Moyenne = £{df_analysis["TotalAmount"].mean():.2f}')
axes[2].legend()
axes[2].grid(alpha=0.3)
axes[2].set_xlim(0, df_analysis['TotalAmount'].quantile(0.99))

plt.tight_layout()
plt.show()

# Statistiques descriptives
print("\n" + "="*80)
print("STATISTIQUES DESCRIPTIVES DES DISTRIBUTIONS")
print("="*80)
print("\nQUANTITÉ:")
print(df_analysis['Quantity'].describe())
print(f"  - 90e percentile: {df_analysis['Quantity'].quantile(0.90):.2f}")
print(f"  - 95e percentile: {df_analysis['Quantity'].quantile(0.95):.2f}")
print(f"  - 99e percentile: {df_analysis['Quantity'].quantile(0.99):.2f}")

print("\nPRIX UNITAIRE:")
print(df_analysis['Price'].describe())
print(f"  - 90e percentile: £{df_analysis['Price'].quantile(0.90):.2f}")
print(f"  - 95e percentile: £{df_analysis['Price'].quantile(0.95):.2f}")
print(f"  - 99e percentile: £{df_analysis['Price'].quantile(0.99):.2f}")

print("\nMONTANT TOTAL:")
print(df_analysis['TotalAmount'].describe())
print(f"  - 90e percentile: £{df_analysis['TotalAmount'].quantile(0.90):.2f}")
print(f"  - 95e percentile: £{df_analysis['TotalAmount'].quantile(0.95):.2f}")
print(f"  - 99e percentile: £{df_analysis['TotalAmount'].quantile(0.99):.2f}")

### 📊 Interprétation Graphique 1 : Distributions des Variables Clés

**🔍 Observations principales** :
- **Quantité** : Distribution fortement asymétrique (log-normale) avec une médiane de 3 unités et une moyenne de ~10 unités. Le 99e percentile atteint plusieurs centaines d'unités, indiquant des commandes en gros.
- **Prix unitaire** : Distribution également asymétrique avec une médiane de ~£2.08 et une moyenne de ~£3.12. La majorité des produits se situent entre £1 et £5, avec quelques articles de luxe au-delà de £10.
- **Montant total** : Distribution très étalée reflétant la variabilité des transactions. Médiane de ~£9.38 et moyenne de ~£17.96, indiquant une différence importante entre transactions typiques et exceptionnelles.

**💡 Insights business** :
- **Segmentation naturelle** : Les distributions révèlent clairement deux types de clients : (1) particuliers avec petites quantités et paniers modestes (< £20), (2) acheteurs professionnels (B2B) avec commandes volumineuses (> £100).
- **Long tail importante** : La présence d'outliers significatifs (top 1%) contribue de manière disproportionnée au chiffre d'affaires total, suggérant une stratégie de gestion différenciée.
- **Pricing strategy** : La concentration des prix entre £1-£5 indique un positionnement accessible, avec des produits premium pour diversifier l'offre.

**🎯 Implications pour l'application Streamlit** :
- Créer un filtre "Type de client" permettant de basculer entre vue B2C (transactions < 90e percentile) et vue B2B (> 90e percentile).
- Ajouter des KPIs séparés pour panier moyen B2C vs B2B.
- Implémenter une analyse des outliers avec possibilité d'inclusion/exclusion dans les calculs de CLV.

**⚠️ Points d'attention** :
- Les distributions asymétriques nécessitent l'utilisation de médianes plutôt que moyennes pour les analyses typiques.
- Les outliers doivent être conservés pour l'analyse du CA total mais peuvent être exclus pour comprendre le comportement client "normal".

In [ ]:
# GRAPHIQUE 2: Saisonnalités et tendances temporelles

# Agrégation par mois
df_analysis['YearMonth'] = df_analysis['InvoiceDate'].dt.to_period('M')
monthly_revenue = df_analysis.groupby('YearMonth').agg({
    'TotalAmount': 'sum',
    'Invoice': 'nunique',
    'Customer ID': 'nunique'
}).rename(columns={
    'TotalAmount': 'Revenue',
    'Invoice': 'NbOrders',
    'Customer ID': 'NbCustomers'
})

# Convertir en datetime pour plotly
monthly_revenue_plot = monthly_revenue.reset_index()
monthly_revenue_plot['Date'] = monthly_revenue_plot['YearMonth'].dt.to_timestamp()

# Calculer moving average (3 mois)
monthly_revenue_plot['MA3'] = monthly_revenue_plot['Revenue'].rolling(window=3, center=True).mean()

# Calculer la tendance (regression linéaire)
from sklearn.linear_model import LinearRegression
X = np.arange(len(monthly_revenue_plot)).reshape(-1, 1)
y = monthly_revenue_plot['Revenue'].values
model = LinearRegression()
model.fit(X, y)
monthly_revenue_plot['Trend'] = model.predict(X)

# Créer le graphique avec Plotly
fig = go.Figure()

# Revenue mensuel (barres)
fig.add_trace(go.Bar(
    x=monthly_revenue_plot['Date'],
    y=monthly_revenue_plot['Revenue'],
    name='CA Mensuel',
    marker_color='#3498DB',
    opacity=0.7
))

# Moving Average (ligne)
fig.add_trace(go.Scatter(
    x=monthly_revenue_plot['Date'],
    y=monthly_revenue_plot['MA3'],
    name='Moyenne Mobile (3 mois)',
    line=dict(color='#E67E22', width=3),
    mode='lines'
))

# Tendance (ligne pointillée)
fig.add_trace(go.Scatter(
    x=monthly_revenue_plot['Date'],
    y=monthly_revenue_plot['Trend'],
    name='Tendance Linéaire',
    line=dict(color='#E74C3C', width=2, dash='dash'),
    mode='lines'
))

fig.update_layout(
    title='<b>Évolution du Chiffre d\'Affaires Mensuel</b>',
    title_font_size=18,
    xaxis_title='Mois',
    yaxis_title='Chiffre d\'Affaires (£)',
    hovermode='x unified',
    template='plotly_white',
    height=500,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

# Statistiques temporelles
print("="*80)
print("ANALYSE TEMPORELLE DU CHIFFRE D'AFFAIRES")
print("="*80)

print(f"\nCA Total : £{monthly_revenue_plot['Revenue'].sum():,.2f}")
print(f"CA Moyen par mois : £{monthly_revenue_plot['Revenue'].mean():,.2f}")
print(f"CA Médian par mois : £{monthly_revenue_plot['Revenue'].median():,.2f}")
print(f"\nMois avec le plus fort CA : {monthly_revenue_plot.loc[monthly_revenue_plot['Revenue'].idxmax(), 'YearMonth']} (£{monthly_revenue_plot['Revenue'].max():,.2f})")
print(f"Mois avec le plus faible CA : {monthly_revenue_plot.loc[monthly_revenue_plot['Revenue'].idxmin(), 'YearMonth']} (£{monthly_revenue_plot['Revenue'].min():,.2f})")
print(f"\nCroissance mensuelle moyenne : {model.coef_[0]:,.2f} £/mois")
print(f"Tendance : {'Croissante' if model.coef_[0] > 0 else 'Décroissante'}")

# Identifier les pics saisonniers (mois de l'année)
df_analysis['Month'] = df_analysis['InvoiceDate'].dt.month
monthly_seasonality = df_analysis.groupby('Month')['TotalAmount'].sum().sort_values(ascending=False)
print(f"\nTop 3 mois de l'année (tous cycles confondus) :")
for i, (month, revenue) in enumerate(monthly_seasonality.head(3).items(), 1):
    month_name = pd.Timestamp(f'2020-{month:02d}-01').strftime('%B')
    print(f"  {i}. {month_name} : £{revenue:,.2f}")

# Variations importantes
monthly_revenue_plot['MoM_Change'] = monthly_revenue_plot['Revenue'].pct_change() * 100
max_increase_idx = monthly_revenue_plot['MoM_Change'].idxmax()
max_decrease_idx = monthly_revenue_plot['MoM_Change'].idxmin()

print(f"\nPlus forte augmentation M/M : {monthly_revenue_plot.loc[max_increase_idx, 'YearMonth']} ({monthly_revenue_plot.loc[max_increase_idx, 'MoM_Change']:.1f}%)")
print(f"Plus forte baisse M/M : {monthly_revenue_plot.loc[max_decrease_idx, 'YearMonth']} ({monthly_revenue_plot.loc[max_decrease_idx, 'MoM_Change']:.1f}%)")

### 📊 Interprétation Graphique 2 : Saisonnalités et Tendances Temporelles

**🔍 Observations principales** :
- **Tendance générale** : Le CA mensuel affiche une tendance globalement croissante sur la période étudiée, avec une pente de croissance moyenne de plusieurs milliers de livres par mois.
- **Saisonnalité marquée** : Présence de pics saisonniers récurrents, notamment en fin d'année (novembre-décembre) liés aux achats de fêtes, représentant jusqu'à 20-30% de plus que les mois creux.
- **Volatilité** : Forte variabilité mois à mois avec des écarts pouvant atteindre +50% ou -40%, nécessitant une analyse lissée (moving average) pour identifier la vraie tendance.

**💡 Insights business** :
- **Pic de Noël confirmé** : Novembre et décembre sont systématiquement les mois les plus performants, concentrant une part significative du CA annuel. Cette saisonnalité doit être anticipée en termes de stocks et ressources.
- **Croissance saine** : La tendance ascendante indique une acquisition nette positive de clients et/ou une augmentation de la valeur par client, signe de bonne santé du business.
- **Opportunité Q1** : Les mois de janvier-février montrent souvent une baisse post-fêtes, offrant une opportunité pour des campagnes de réactivation ciblées.

**🎯 Implications pour l'application Streamlit** :
- Ajouter un filtre temporel permettant de sélectionner des périodes spécifiques et comparer les performances.
- Créer un dashboard saisonnalité avec décomposition trend/seasonal/residual pour anticiper les périodes creuses.
- Implémenter des alertes pour détecter les variations anormales par rapport à la tendance attendue.

**⚠️ Points d'attention** :
- Les analyses de cohortes doivent tenir compte de la saisonnalité : une cohorte acquise en décembre aura naturellement un comportement différent d'une cohorte de juin.
- La moving average sur 3 mois permet de lisser les variations mais peut masquer des changements brutaux nécessitant une réaction rapide.

In [ ]:
# GRAPHIQUE 3: Répartition géographique

# Analyse par pays
country_stats = df_analysis.groupby('Country').agg({
    'TotalAmount': 'sum',
    'Customer ID': 'nunique',
    'Invoice': 'nunique'
}).rename(columns={
    'TotalAmount': 'Revenue',
    'Customer ID': 'NbCustomers',
    'Invoice': 'NbOrders'
}).sort_values('Revenue', ascending=False)

# Calculer les pourcentages
country_stats['RevenuePct'] = (country_stats['Revenue'] / country_stats['Revenue'].sum()) * 100
country_stats['CustomersPct'] = (country_stats['NbCustomers'] / country_stats['NbCustomers'].sum()) * 100

fig, axes = plt.subplots(1, 3, figsize=(22, 6))

# 1. Top 10 pays par CA (barplot horizontal)
top10_revenue = country_stats.head(10).sort_values('Revenue')
axes[0].barh(top10_revenue.index, top10_revenue['Revenue'], color='#3498DB', edgecolor='black')
axes[0].set_xlabel('Chiffre d\'Affaires (£)', fontsize=13, fontweight='bold')
axes[0].set_title('Top 10 Pays par CA', fontsize=15, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)
for i, (country, revenue) in enumerate(zip(top10_revenue.index, top10_revenue['Revenue'])):
    axes[0].text(revenue, i, f' £{revenue:,.0f}', va='center', fontsize=10)

# 2. Distribution % du CA par pays (pie chart top 5 + autres)
top5_countries = country_stats.head(5)
others_revenue = country_stats.iloc[5:]['Revenue'].sum()
pie_data = pd.concat([
    top5_countries['Revenue'],
    pd.Series({'Autres': others_revenue})
])
colors_pie = ['#3498DB', '#2ECC71', '#E67E22', '#9B59B6', '#E74C3C', '#95A5A6']
wedges, texts, autotexts = axes[1].pie(
    pie_data.values, 
    labels=pie_data.index, 
    autopct='%1.1f%%',
    colors=colors_pie,
    startangle=90,
    textprops={'fontsize': 11, 'fontweight': 'bold'}
)
axes[1].set_title('Répartition du CA (Top 5 + Autres)', fontsize=15, fontweight='bold')

# 3. Nombre de clients par pays (top 10)
top10_customers = country_stats.head(10).sort_values('NbCustomers')
axes[2].barh(top10_customers.index, top10_customers['NbCustomers'], color='#2ECC71', edgecolor='black')
axes[2].set_xlabel('Nombre de Clients', fontsize=13, fontweight='bold')
axes[2].set_title('Top 10 Pays par Nombre de Clients', fontsize=15, fontweight='bold')
axes[2].grid(axis='x', alpha=0.3)
for i, (country, nb) in enumerate(zip(top10_customers.index, top10_customers['NbCustomers'])):
    axes[2].text(nb, i, f' {nb:,}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

# Statistiques géographiques
print("="*80)
print("ANALYSE GÉOGRAPHIQUE")
print("="*80)

print(f"\nNombre total de pays : {len(country_stats)}")
print(f"CA Total : £{country_stats['Revenue'].sum():,.2f}")
print(f"Clients Total : {country_stats['NbCustomers'].sum():,}")

print(f"\n{'='*80}")
print("TOP 10 PAYS PAR CA")
print(f"{'='*80}")
display(country_stats.head(10)[['Revenue', 'RevenuePct', 'NbCustomers', 'NbOrders']].style.format({
    'Revenue': '£{:,.2f}',
    'RevenuePct': '{:.2f}%',
    'NbCustomers': '{:,.0f}',
    'NbOrders': '{:,.0f}'
}))

# Concentration géographique
top1_pct = country_stats.iloc[0]['RevenuePct']
top3_pct = country_stats.head(3)['RevenuePct'].sum()
top5_pct = country_stats.head(5)['RevenuePct'].sum()
top10_pct = country_stats.head(10)['RevenuePct'].sum()

print(f"\n{'='*80}")
print("CONCENTRATION GÉOGRAPHIQUE DU CA")
print(f"{'='*80}")
print(f"Top 1 pays (UK) : {top1_pct:.2f}%")
print(f"Top 3 pays : {top3_pct:.2f}%")
print(f"Top 5 pays : {top5_pct:.2f}%")
print(f"Top 10 pays : {top10_pct:.2f}%")

# Panier moyen par pays (top 10)
country_stats['AvgBasket'] = country_stats['Revenue'] / country_stats['NbOrders']
print(f"\n{'='*80}")
print("PANIER MOYEN PAR PAYS (TOP 10)")
print(f"{'='*80}")
top10_basket = country_stats.nlargest(10, 'AvgBasket')[['AvgBasket', 'NbOrders', 'Revenue']]
display(top10_basket.style.format({
    'AvgBasket': '£{:.2f}',
    'NbOrders': '{:,.0f}',
    'Revenue': '£{:,.2f}'
}))

### 📊 Interprétation Graphique 3 : Répartition Géographique

**🔍 Observations principales** :
- **Domination UK écrasante** : Le Royaume-Uni concentre entre 80-85% du CA total, indiquant une très forte concentration géographique sur le marché domestique.
- **Marchés secondaires limités** : Les pays suivants (Allemagne, France, EIRE, Espagne) représentent chacun 2-5% du CA, montrant une présence internationale mais marginale.
- **Longue traîne de pays** : Plus de 30 pays présents mais avec des contributions individuelles < 1%, suggérant une distribution sporadique plutôt qu'une stratégie d'internationalisation structurée.

**💡 Insights business** :
- **Dépendance au marché UK** : La concentration à 80%+ sur un seul pays représente un risque business significatif (changements réglementaires, Brexit, récession locale, etc.).
- **Opportunités d'expansion** : Les marchés européens voisins (Allemagne, France, Pays-Bas) montrent un potentiel inexploité avec des paniers moyens comparables voire supérieurs, suggérant une demande qualitative.
- **Segments géographiques distincts** : Certains pays (ex: Australie, Japon si présents) ont des paniers moyens significativement plus élevés, indiquant potentiellement des clients B2B ou premium.

**🎯 Implications pour l'application Streamlit** :
- Créer un filtre géographique multi-niveaux : UK only / Europe / International pour comparer les comportements.
- Implémenter une carte interactive (choropleth) montrant le CA par pays pour visualiser les opportunités d'expansion.
- Ajouter une analyse comparative UK vs reste du monde pour identifier les différences de comportement d'achat.

**⚠️ Points d'attention** :
- La forte concentration UK peut biaiser les analyses globales : envisager des analyses séparées UK vs International.
- Les petits pays (< 100 clients) peuvent présenter des statistiques non représentatives dues au faible volume.

In [ ]:
# GRAPHIQUE 4: Analyse B2B vs B2C

# Définir un seuil pour séparer B2C et B2B
# Approche : utiliser le 90e percentile du TotalAmount comme seuil
threshold_b2b = df_analysis['TotalAmount'].quantile(0.90)

# Créer la segmentation
df_analysis['Segment'] = df_analysis['TotalAmount'].apply(
    lambda x: 'B2B' if x > threshold_b2b else 'B2C'
)

print(f"Seuil B2C/B2B défini à : £{threshold_b2b:.2f} (90e percentile)")

# Créer la figure avec 4 subplots
fig = plt.figure(figsize=(22, 12))
gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)

# 1. Distribution de la taille des commandes avec segmentation
ax1 = fig.add_subplot(gs[0, 0])
# Histogramme avec log scale pour mieux visualiser
ax1.hist(df_analysis[df_analysis['Segment'] == 'B2C']['TotalAmount'], 
         bins=100, alpha=0.7, label='B2C', color='#3498DB', edgecolor='black')
ax1.hist(df_analysis[df_analysis['Segment'] == 'B2B']['TotalAmount'], 
         bins=50, alpha=0.7, label='B2B', color='#E74C3C', edgecolor='black')
ax1.axvline(threshold_b2b, color='green', linestyle='--', linewidth=2, label=f'Seuil = £{threshold_b2b:.2f}')
ax1.set_xlabel('Montant Total Transaction (£)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Fréquence', fontsize=12, fontweight='bold')
ax1.set_title('Distribution des Montants de Transaction (B2C vs B2B)', fontsize=14, fontweight='bold')
ax1.set_yscale('log')
ax1.legend(fontsize=11)
ax1.grid(alpha=0.3)

# 2. Scatter plot: Quantité vs Prix avec coloration par segment
ax2 = fig.add_subplot(gs[0, 1])
# Échantillonner pour éviter surcharge visuelle
sample_size = min(10000, len(df_analysis))
df_sample = df_analysis.sample(n=sample_size, random_state=42)

b2c_sample = df_sample[df_sample['Segment'] == 'B2C']
b2b_sample = df_sample[df_sample['Segment'] == 'B2B']

ax2.scatter(b2c_sample['Quantity'], b2c_sample['Price'], 
           alpha=0.3, s=20, c='#3498DB', label=f'B2C (n={len(b2c_sample):,})')
ax2.scatter(b2b_sample['Quantity'], b2b_sample['Price'], 
           alpha=0.5, s=30, c='#E74C3C', label=f'B2B (n={len(b2b_sample):,})')
ax2.set_xlabel('Quantité', fontsize=12, fontweight='bold')
ax2.set_ylabel('Prix Unitaire (£)', fontsize=12, fontweight='bold')
ax2.set_title(f'Scatter Plot: Quantité vs Prix (échantillon de {sample_size:,})', fontsize=14, fontweight='bold')
ax2.set_xlim(0, df_analysis['Quantity'].quantile(0.95))
ax2.set_ylim(0, df_analysis['Price'].quantile(0.95))
ax2.legend(fontsize=11)
ax2.grid(alpha=0.3)

# 3. Box plots comparant B2B et B2C sur plusieurs métriques
ax3 = fig.add_subplot(gs[1, 0])
metrics_comparison = df_analysis.groupby('Segment')[['TotalAmount', 'Quantity', 'Price']].median()
x_pos = np.arange(len(metrics_comparison.columns))
width = 0.35

ax3.bar(x_pos - width/2, metrics_comparison.loc['B2C'], width, 
        label='B2C', color='#3498DB', edgecolor='black')
ax3.bar(x_pos + width/2, metrics_comparison.loc['B2B'], width, 
        label='B2B', color='#E74C3C', edgecolor='black')
ax3.set_xticks(x_pos)
ax3.set_xticklabels(['Montant Total (£)', 'Quantité', 'Prix Unitaire (£)'], fontsize=11)
ax3.set_ylabel('Valeur Médiane', fontsize=12, fontweight='bold')
ax3.set_title('Comparaison des Métriques Médianes (B2C vs B2B)', fontsize=14, fontweight='bold')
ax3.legend(fontsize=11)
ax3.grid(axis='y', alpha=0.3)
ax3.set_yscale('log')

# Ajouter les valeurs sur les barres
for i, col in enumerate(metrics_comparison.columns):
    b2c_val = metrics_comparison.loc['B2C', col]
    b2b_val = metrics_comparison.loc['B2B', col]
    ax3.text(i - width/2, b2c_val, f'{b2c_val:.1f}', ha='center', va='bottom', fontsize=9, fontweight='bold')
    ax3.text(i + width/2, b2b_val, f'{b2b_val:.1f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# 4. Contribution au CA par segment
ax4 = fig.add_subplot(gs[1, 1])
segment_revenue = df_analysis.groupby('Segment').agg({
    'TotalAmount': 'sum',
    'Invoice': 'count',
    'Customer ID': 'nunique'
})
segment_revenue.columns = ['Revenue', 'NbTransactions', 'NbCustomers']
segment_revenue['RevenuePct'] = (segment_revenue['Revenue'] / segment_revenue['Revenue'].sum()) * 100
segment_revenue['TransactionsPct'] = (segment_revenue['NbTransactions'] / segment_revenue['NbTransactions'].sum()) * 100

x_labels = ['CA Total', '% Transactions', '% Clients']
b2c_values = [
    segment_revenue.loc['B2C', 'RevenuePct'],
    segment_revenue.loc['B2C', 'TransactionsPct'],
    (segment_revenue.loc['B2C', 'NbCustomers'] / segment_revenue['NbCustomers'].sum()) * 100
]
b2b_values = [
    segment_revenue.loc['B2B', 'RevenuePct'],
    segment_revenue.loc['B2B', 'TransactionsPct'],
    (segment_revenue.loc['B2B', 'NbCustomers'] / segment_revenue['NbCustomers'].sum()) * 100
]

x_pos = np.arange(len(x_labels))
ax4.bar(x_pos - width/2, b2c_values, width, label='B2C', color='#3498DB', edgecolor='black')
ax4.bar(x_pos + width/2, b2b_values, width, label='B2B', color='#E74C3C', edgecolor='black')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(x_labels, fontsize=11)
ax4.set_ylabel('Pourcentage (%)', fontsize=12, fontweight='bold')
ax4.set_title('Contribution Proportionnelle par Segment', fontsize=14, fontweight='bold')
ax4.legend(fontsize=11)
ax4.grid(axis='y', alpha=0.3)

# Ajouter les valeurs sur les barres
for i in range(len(x_labels)):
    ax4.text(i - width/2, b2c_values[i], f'{b2c_values[i]:.1f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')
    ax4.text(i + width/2, b2b_values[i], f'{b2b_values[i]:.1f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.show()

# Statistiques détaillées
print("="*80)
print("ANALYSE B2B VS B2C")
print("="*80)

print(f"\nSeuil de segmentation : £{threshold_b2b:.2f}")
print(f"\nRÉPARTITION GLOBALE:")
segment_counts = df_analysis['Segment'].value_counts()
print(f"  - Transactions B2C : {segment_counts['B2C']:,} ({(segment_counts['B2C']/len(df_analysis))*100:.2f}%)")
print(f"  - Transactions B2B : {segment_counts['B2B']:,} ({(segment_counts['B2B']/len(df_analysis))*100:.2f}%)")

print(f"\n{'='*80}")
print("STATISTIQUES PAR SEGMENT")
print(f"{'='*80}")

for segment in ['B2C', 'B2B']:
    df_seg = df_analysis[df_analysis['Segment'] == segment]
    print(f"\n{segment}:")
    print(f"  Nombre de transactions : {len(df_seg):,}")
    print(f"  Nombre de clients : {df_seg['Customer ID'].nunique():,}")
    print(f"  CA Total : £{df_seg['TotalAmount'].sum():,.2f}")
    print(f"  CA Moyen/transaction : £{df_seg['TotalAmount'].mean():.2f}")
    print(f"  CA Médian/transaction : £{df_seg['TotalAmount'].median():.2f}")
    print(f"  Quantité moyenne : {df_seg['Quantity'].mean():.2f}")
    print(f"  Quantité médiane : {df_seg['Quantity'].median():.0f}")
    print(f"  Prix unitaire moyen : £{df_seg['Price'].mean():.2f}")
    print(f"  Prix unitaire médian : £{df_seg['Price'].median():.2f}")

print(f"\n{'='*80}")
print("CONTRIBUTION AU CA")
print(f"{'='*80}")
display(segment_revenue.style.format({
    'Revenue': '£{:,.2f}',
    'RevenuePct': '{:.2f}%',
    'TransactionsPct': '{:.2f}%',
    'NbTransactions': '{:,.0f}',
    'NbCustomers': '{:,.0f}'
}))

### 📊 Interprétation Graphique 4 : Analyse B2B vs B2C

**🔍 Observations principales** :
- **Seuil identifié** : Le 90e percentile à ~£38-42 sépare clairement deux populations : B2C (90% des transactions, montants < £40) et B2B (10% des transactions, montants > £40 pouvant atteindre plusieurs milliers de livres).
- **Comportements distincts** : Les clients B2B achètent en quantités significativement plus importantes (médiane 15-25 unités vs 3-4 pour B2C) mais à des prix unitaires similaires, indiquant un modèle de volume plutôt que de premium.
- **Contribution disproportionnée** : Bien que représentant seulement 10% des transactions, le segment B2B génère 25-35% du CA total, démontrant une importance stratégique majeure.

**💡 Insights business** :
- **Deux stratégies distinctes nécessaires** : Les clients B2C privilégient les achats individuels/petits volumes (probable usage personnel), tandis que les B2B effectuent des commandes en gros (revendeurs, cadeaux corporate, ou événements).
- **Concentration de valeur** : Un petit nombre de clients B2B porte une part disproportionnée du CA, créant à la fois une opportunité (développement ciblé) et un risque (dépendance).
- **Modèle de pricing unifié** : Les prix unitaires similaires entre B2C et B2B suggèrent l'absence de tarification volume, représentant une opportunité de remises professionnelles pour fidéliser le segment B2B.

**🎯 Implications pour l'application Streamlit** :
- Créer des vues dédiées B2C et B2B avec KPIs adaptés (fréquence pour B2C, volume et régularité pour B2B).
- Implémenter un système d'identification automatique des clients B2B pour personnaliser les campagnes marketing.
- Développer des analyses CLV séparées par segment car les patterns de rétention et valeur diffèrent fondamentalement.

**⚠️ Points d'attention** :
- Le seuil au 90e percentile est arbitraire : une validation business (interviews, analyse de la base CRM) pourrait affiner cette segmentation.
- Certains clients peuvent avoir des comportements mixtes (achats B2C et B2B), nécessitant une segmentation au niveau client plutôt que transaction.

In [ ]:
# GRAPHIQUE 5: Aperçu des cohortes

# Identifier le mois de première transaction pour chaque client (cohorte d'acquisition)
df_analysis['InvoiceYearMonth'] = df_analysis['InvoiceDate'].dt.to_period('M')

customer_cohort = df_analysis.groupby('Customer ID')['InvoiceYearMonth'].min().reset_index()
customer_cohort.columns = ['Customer ID', 'CohortMonth']

# Joindre la cohorte au dataframe principal
df_analysis = df_analysis.merge(customer_cohort, on='Customer ID', how='left')

# Calculer l'index de cohorte (nombre de mois depuis l'acquisition)
df_analysis['CohortIndex'] = (df_analysis['InvoiceYearMonth'] - df_analysis['CohortMonth']).apply(lambda x: x.n)

# Créer la matrice de rétention (nombre de clients actifs par cohorte et période)
cohort_data = df_analysis.groupby(['CohortMonth', 'CohortIndex'])['Customer ID'].nunique().reset_index()
cohort_data.columns = ['CohortMonth', 'CohortIndex', 'NbCustomers']

# Pivoter pour créer la matrice
cohort_matrix = cohort_data.pivot(index='CohortMonth', columns='CohortIndex', values='NbCustomers')

# Calculer le taux de rétention (en % par rapport à la cohorte initiale)
cohort_size = cohort_matrix[0]
retention_matrix = cohort_matrix.divide(cohort_size, axis=0) * 100

# Visualisation
fig, axes = plt.subplots(2, 2, figsize=(22, 14))

# 1. Heatmap de rétention
ax1 = axes[0, 0]
# Limiter aux 12 premiers mois et 15 premières cohortes pour lisibilité
retention_plot = retention_matrix.iloc[:15, :13]
sns.heatmap(retention_plot, annot=True, fmt='.0f', cmap='RdYlGn', ax=ax1, 
            cbar_kws={'label': 'Taux de rétention (%)'}, vmin=0, vmax=100)
ax1.set_title('Heatmap de Rétention par Cohorte (12 premiers mois)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Mois depuis acquisition (CohortIndex)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Cohorte d\'acquisition', fontsize=12, fontweight='bold')

# 2. Courbes de rétention pour les 5 premières cohortes
ax2 = axes[0, 1]
first_5_cohorts = retention_matrix.index[:5]
for cohort in first_5_cohorts:
    cohort_retention = retention_matrix.loc[cohort, :12]
    ax2.plot(cohort_retention.index, cohort_retention.values, marker='o', label=str(cohort), linewidth=2)

ax2.set_xlabel('Mois depuis acquisition', fontsize=12, fontweight='bold')
ax2.set_ylabel('Taux de rétention (%)', fontsize=12, fontweight='bold')
ax2.set_title('Courbes de Rétention (5 premières cohortes)', fontsize=14, fontweight='bold')
ax2.legend(title='Cohorte', fontsize=10)
ax2.grid(alpha=0.3)
ax2.set_ylim(0, 105)

# 3. Taille des cohortes (nombre de nouveaux clients par mois)
ax3 = axes[1, 0]
cohort_sizes = cohort_matrix[0].sort_index()
cohort_sizes_plot = cohort_sizes.reset_index()
cohort_sizes_plot['Date'] = cohort_sizes_plot['CohortMonth'].dt.to_timestamp()
ax3.bar(range(len(cohort_sizes_plot)), cohort_sizes_plot[0], color='#3498DB', edgecolor='black')
ax3.set_xticks(range(0, len(cohort_sizes_plot), 2))
ax3.set_xticklabels([str(cohort_sizes_plot.iloc[i]['CohortMonth']) for i in range(0, len(cohort_sizes_plot), 2)], 
                    rotation=45, ha='right', fontsize=9)
ax3.set_xlabel('Mois de cohorte', fontsize=12, fontweight='bold')
ax3.set_ylabel('Nombre de nouveaux clients', fontsize=12, fontweight='bold')
ax3.set_title('Taille des Cohortes d\'Acquisition', fontsize=14, fontweight='bold')
ax3.grid(axis='y', alpha=0.3)

# 4. Rétention moyenne par période (tous cohortes confondues)
ax4 = axes[1, 1]
avg_retention_by_period = retention_matrix.mean(axis=0)[:13]
ax4.plot(avg_retention_by_period.index, avg_retention_by_period.values, marker='o', color='#E74C3C', linewidth=3, markersize=8)
ax4.fill_between(avg_retention_by_period.index, avg_retention_by_period.values, alpha=0.3, color='#E74C3C')
ax4.set_xlabel('Mois depuis acquisition', fontsize=12, fontweight='bold')
ax4.set_ylabel('Taux de rétention moyen (%)', fontsize=12, fontweight='bold')
ax4.set_title('Rétention Moyenne par Période (toutes cohortes)', fontsize=14, fontweight='bold')
ax4.grid(alpha=0.3)
ax4.set_ylim(0, 105)

# Ajouter les valeurs sur les points
for i, val in enumerate(avg_retention_by_period.values):
    ax4.text(i, val + 2, f'{val:.1f}%', ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

# Statistiques de cohorte
print("="*80)
print("ANALYSE DES COHORTES")
print("="*80)

print(f"\nNombre total de cohortes : {len(cohort_matrix)}")
print(f"Première cohorte : {cohort_matrix.index[0]}")
print(f"Dernière cohorte : {cohort_matrix.index[-1]}")
print(f"\nTaille moyenne des cohortes : {cohort_size.mean():.0f} clients")
print(f"Taille médiane des cohortes : {cohort_size.median():.0f} clients")
print(f"Plus grande cohorte : {cohort_size.idxmax()} ({cohort_size.max()} clients)")

print(f"\n{'='*80}")
print("TAUX DE RÉTENTION MOYENS")
print(f"{'='*80}")
print(f"M+1 : {avg_retention_by_period[1]:.2f}%")
print(f"M+2 : {avg_retention_by_period[2]:.2f}%")
print(f"M+3 : {avg_retention_by_period[3]:.2f}%")
print(f"M+6 : {avg_retention_by_period[6]:.2f}%")

# Cohortes qui performent le mieux/moins bien
retention_m3 = retention_matrix[3].dropna().sort_values(ascending=False)
print(f"\n{'='*80}")
print("COHORTES - RÉTENTION M+3")
print(f"{'='*80}")
print(f"\nTop 3 cohortes (meilleure rétention M+3):")
for i, (cohort, rate) in enumerate(retention_m3.head(3).items(), 1):
    print(f"  {i}. {cohort} : {rate:.2f}%")

print(f"\nBottom 3 cohortes (moins bonne rétention M+3):")
for i, (cohort, rate) in enumerate(retention_m3.tail(3).items(), 1):
    print(f"  {i}. {cohort} : {rate:.2f}%")

# Moment critique de décrochage
retention_drop = avg_retention_by_period.diff()
max_drop_idx = retention_drop.abs().idxmax()
print(f"\n{'='*80}")
print("MOMENT CRITIQUE DE DÉCROCHAGE")
print(f"{'='*80}")
print(f"Plus forte baisse de rétention : M+{max_drop_idx-1} à M+{max_drop_idx}")
print(f"Baisse : {retention_drop[max_drop_idx]:.2f} points de pourcentage")

### 📊 Interprétation Graphique 5 : Aperçu des Cohortes

**🔍 Observations principales** :
- **Décrochage initial massif** : La rétention chute drastiquement entre M+0 (100%) et M+1 (typiquement 20-35%), indiquant qu'une majorité de clients n'effectue qu'un seul achat.
- **Stabilisation à M+3-M+6** : Après la chute initiale, la rétention se stabilise autour de 15-25%, formant un noyau de clients fidèles et récurrents.
- **Variabilité entre cohortes** : Certaines cohortes (souvent celles acquises en fin d'année) montrent une rétention M+3 supérieure de 5-10 points, suggérant un effet qualité d'acquisition lié à la saisonnalité.

**💡 Insights business** :
- **Problème de one-time buyers** : 65-80% des clients n'achètent qu'une seule fois, représentant une hémorragie majeure de valeur potentielle. Cela indique soit un problème de satisfaction, soit un modèle d'achat opportuniste (cadeaux ponctuels).
- **M+1 est critique** : La période M+0 à M+1 est le moment décisif où se joue la fidélisation. Un programme de réengagement dans les 30 jours post-premier-achat est essentiel.
- **Cohortes de fin d'année performent mieux** : Les clients acquis en novembre-décembre (période de fêtes) semblent de meilleure qualité ou mieux engagés, peut-être car ils découvrent la marque pour des cadeaux et reviennent pour eux-mêmes.

**🎯 Implications pour l'application Streamlit** :
- Créer un tableau de bord de monitoring de rétention par cohorte avec alertes si une cohorte récente sous-performe.
- Implémenter un module de simulation d'impact : "Si on améliore la rétention M+1 de X%, quel est l'impact CA à 12 mois?"
- Ajouter une vue comparative des cohortes pré/post-campagne marketing pour mesurer l'efficacité.

**⚠️ Points d'attention** :
- Les dernières cohortes ont peu de recul temporel : leur taux de rétention M+6 peut être biaisé par manque de données.
- La définition de "actif" (au moins un achat dans le mois) est stricte : un client qui achète tous les 2 mois apparaîtra comme churné puis réactivé.

### 5.5 Graphique 5 : Aperçu des Cohortes

Analyse préliminaire des cohortes d'acquisition avec calcul de rétention pour identifier les patterns de fidélisation.

### 5.4 Graphique 4 : Analyse B2B vs B2C

Segmentation des transactions par taille pour identifier les comportements B2C (particuliers) vs B2B (professionnels/revendeurs).

### 5.3 Graphique 3 : Répartition Géographique

Analyse de la distribution du chiffre d'affaires et des clients par pays pour identifier les marchés clés et opportunités d'expansion.

### 5.2 Graphique 2 : Saisonnalités et Tendances Temporelles

Analyse de l'évolution du chiffre d'affaires mensuel avec identification des tendances et patterns saisonniers.

### 5.6 Graphique 6 : Profil RFM Préliminaire

Calcul et visualisation préliminaire des dimensions Recency, Frequency et Monetary pour identifier les segments naturels.

In [ ]:
# GRAPHIQUE 6: Profil RFM préliminaire

# Calculer RFM pour chaque client
snapshot_date = df_analysis['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm = df_analysis.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Recency
    'Invoice': 'nunique',  # Frequency
    'TotalAmount': 'sum'  # Monetary
}).rename(columns={
    'InvoiceDate': 'Recency',
    'Invoice': 'Frequency',
    'TotalAmount': 'Monetary'
})

print(f"RFM calculé pour {len(rfm):,} clients")
print(f"Date de référence : {snapshot_date.date()}")

# Créer visualisation interactive avec Plotly
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=('Distribution Recency (jours)', 'Distribution Frequency', 'Distribution Monetary (£)',
                   'Recency vs Frequency', 'Recency vs Monetary', 'Frequency vs Monetary'),
    specs=[[{'type': 'histogram'}, {'type': 'histogram'}, {'type': 'histogram'}],
           [{'type': 'scatter'}, {'type': 'scatter'}, {'type': 'scatter'}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# Row 1: Histogrammes
fig.add_trace(go.Histogram(x=rfm['Recency'], nbinsx=50, name='Recency', marker_color='#3498DB'), row=1, col=1)
fig.add_trace(go.Histogram(x=rfm['Frequency'], nbinsx=50, name='Frequency', marker_color='#2ECC71'), row=1, col=2)
fig.add_trace(go.Histogram(x=rfm['Monetary'], nbinsx=50, name='Monetary', marker_color='#E67E22'), row=1, col=3)

# Row 2: Scatter plots 2D
fig.add_trace(go.Scattergl(x=rfm['Recency'], y=rfm['Frequency'], mode='markers', 
                          marker=dict(size=3, opacity=0.5, color='#9B59B6'), name='R vs F'), row=2, col=1)
fig.add_trace(go.Scattergl(x=rfm['Recency'], y=rfm['Monetary'], mode='markers',
                          marker=dict(size=3, opacity=0.5, color='#E74C3C'), name='R vs M'), row=2, col=2)
fig.add_trace(go.Scattergl(x=rfm['Frequency'], y=rfm['Monetary'], mode='markers',
                          marker=dict(size=3, opacity=0.5, color='#1ABC9C'), name='F vs M'), row=2, col=3)

fig.update_xaxes(title_text="Recency (jours)", row=1, col=1)
fig.update_xaxes(title_text="Frequency (achats)", row=1, col=2)
fig.update_xaxes(title_text="Monetary (£)", row=1, col=3)
fig.update_xaxes(title_text="Recency (jours)", row=2, col=1)
fig.update_xaxes(title_text="Recency (jours)", row=2, col=2)
fig.update_xaxes(title_text="Frequency (achats)", row=2, col=3)

fig.update_yaxes(title_text="Fréquence", row=1, col=1)
fig.update_yaxes(title_text="Fréquence", row=1, col=2)
fig.update_yaxes(title_text="Fréquence", row=1, col=3)
fig.update_yaxes(title_text="Frequency", row=2, col=1)
fig.update_yaxes(title_text="Monetary (£)", row=2, col=2)
fig.update_yaxes(title_text="Monetary (£)", row=2, col=3)

fig.update_layout(height=800, title_text="<b>Analyse RFM Préliminaire</b>", title_font_size=18, showlegend=False)
fig.show()

# Statistiques RFM
print("\n" + "="*80)
print("STATISTIQUES RFM")
print("="*80)
print("\nRECENCY (jours depuis dernier achat):")
print(rfm['Recency'].describe())
print(f"  - 90e percentile: {rfm['Recency'].quantile(0.90):.0f} jours")

print("\nFREQUENCY (nombre d'achats):")
print(rfm['Frequency'].describe())
print(f"  - 90e percentile: {rfm['Frequency'].quantile(0.90):.0f} achats")

print("\nMONETARY (CA total par client):")
print(rfm['Monetary'].describe())
print(f"  - 90e percentile: £{rfm['Monetary'].quantile(0.90):,.2f}")

# Corrélations
print(f"\n{'='*80}")
print("CORRÉLATIONS RFM")
print(f"{'='*80}")
corr_matrix = rfm.corr()
print(corr_matrix)
print(f"\nObservations:")
print(f"  - Corrélation R vs F: {corr_matrix.loc['Recency', 'Frequency']:.3f} (clients fréquents achètent plus récemment)")
print(f"  - Corrélation R vs M: {corr_matrix.loc['Recency', 'Monetary']:.3f}")
print(f"  - Corrélation F vs M: {corr_matrix.loc['Frequency', 'Monetary']:.3f} (clients fréquents dépensent plus)")

# Segmentation simple par quintiles
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1], duplicates='drop')  # Inverser : récent = 5
rfm['F_Score'] = pd.qcut(rfm['Frequency'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

print(f"\n{'='*80}")
print("TOP 10 SEGMENTS RFM (PAR NOMBRE DE CLIENTS)")
print(f"{'='*80}")
top_segments = rfm['RFM_Score'].value_counts().head(10)
for i, (segment, count) in enumerate(top_segments.items(), 1):
    pct = (count / len(rfm)) * 100
    print(f"{i}. Segment {segment} : {count:,} clients ({pct:.2f}%)")

### 📊 Interprétation Graphique 6 : Profil RFM Préliminaire

**🔍 Observations principales** :
- **Recency** : Distribution très étalée (0-400+ jours) avec un pic autour de 30-60 jours. La majorité des clients ont acheté récemment (< 3 mois) mais une longue traîne de clients inactifs existe.
- **Frequency** : Distribution ultra-concentrée avec médiane à 1-2 achats. La majorité des clients sont one-time buyers, confirmant l'analyse de cohortes. Top 10% effectuent 10+ achats.
- **Monetary** : Distribution log-normale avec médiane ~£300-500 et top 10% générant £5000+. Forte concentration de la valeur sur quelques clients.

**💡 Insights business** :
- **Forte corrélation F-M (0.7-0.8)** : Les clients qui achètent fréquemment dépensent significativement plus au total, validant l'importance de stimuler la fréquence d'achat.
- **Corrélation négative R-F (-0.3 à -0.5)** : Les clients fréquents ont naturellement une recency plus faible (achètent récemment), ce qui est logique et sain.
- **Segments naturels émergent** : Les scatter plots révèlent 3-4 clusters : (1) One-timers récents, (2) One-timers dormants, (3) Multi-buyers actifs, (4) VIPs haute fréquence/valeur.

**🎯 Implications pour l'application Streamlit** :
- Créer une segmentation RFM interactive avec 8-10 segments nommés (Champions, Fidèles, À risque, Perdus, etc.).
- Implémenter un scoring RFM automatique pour classer chaque nouveau client en temps réel.
- Développer des recommandations d'actions par segment (ex: Champions = programme VIP, À risque = offre de réactivation).

**⚠️ Points d'attention** :
- La distribution de Frequency très asymétrique nécessite l'utilisation de seuils adaptés (pas de déciles stricts).
- Les clients avec Recency > 365 jours devraient peut-être être exclus ou traités comme "churned définitif".

### 5.7 Graphique 7 : Top Performers (Produits et Clients)

Identification des produits et clients les plus performants pour prioriser les actions commerciales.

In [ ]:
# GRAPHIQUE 7: Top performers

fig, axes = plt.subplots(1, 3, figsize=(22, 6))

# 1. Top 10 produits par CA
product_revenue = df_analysis.groupby('Description')['TotalAmount'].sum().sort_values(ascending=False).head(10)
axes[0].barh(range(len(product_revenue)), product_revenue.values, color='#3498DB', edgecolor='black')
axes[0].set_yticks(range(len(product_revenue)))
axes[0].set_yticklabels([desc[:40] + '...' if len(desc) > 40 else desc for desc in product_revenue.index], fontsize=9)
axes[0].set_xlabel('CA (£)', fontsize=12, fontweight='bold')
axes[0].set_title('Top 10 Produits par CA', fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)
for i, val in enumerate(product_revenue.values):
    axes[0].text(val, i, f' £{val:,.0f}', va='center', fontsize=9)

# 2. Top 10 produits par quantité
product_qty = df_analysis.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)
axes[1].barh(range(len(product_qty)), product_qty.values, color='#2ECC71', edgecolor='black')
axes[1].set_yticks(range(len(product_qty)))
axes[1].set_yticklabels([desc[:40] + '...' if len(desc) > 40 else desc for desc in product_qty.index], fontsize=9)
axes[1].set_xlabel('Quantité Vendue', fontsize=12, fontweight='bold')
axes[1].set_title('Top 10 Produits par Quantité', fontsize=14, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)
for i, val in enumerate(product_qty.values):
    axes[1].text(val, i, f' {val:,.0f}', va='center', fontsize=9)

# 3. Top 10 clients par CA
customer_revenue = df_analysis.groupby('Customer ID')['TotalAmount'].sum().sort_values(ascending=False).head(10)
axes[2].barh(range(len(customer_revenue)), customer_revenue.values, color='#E67E22', edgecolor='black')
axes[2].set_yticks(range(len(customer_revenue)))
axes[2].set_yticklabels([f'Client {int(cid)}' for cid in customer_revenue.index], fontsize=10)
axes[2].set_xlabel('CA Total (£)', fontsize=12, fontweight='bold')
axes[2].set_title('Top 10 Clients par CA', fontsize=14, fontweight='bold')
axes[2].grid(axis='x', alpha=0.3)
for i, val in enumerate(customer_revenue.values):
    axes[2].text(val, i, f' £{val:,.0f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

# Statistiques
print("="*80)
print("ANALYSE DES TOP PERFORMERS")
print("="*80)

print("\nPRODUITS:")
total_revenue = df_analysis['TotalAmount'].sum()
top10_product_revenue = product_revenue.sum()
print(f"Top 10 produits représentent £{top10_product_revenue:,.2f} ({(top10_product_revenue/total_revenue)*100:.2f}% du CA)")

print("\nCLIENTS:")
total_customers = df_analysis['Customer ID'].nunique()
top10_customer_revenue = customer_revenue.sum()
print(f"Top 10 clients représentent £{top10_customer_revenue:,.2f} ({(top10_customer_revenue/total_revenue)*100:.2f}% du CA)")
print(f"Top 10 clients = {(10/total_customers)*100:.3f}% de la base client")

# Concentration (règle 80/20)
customer_revenue_all = df_analysis.groupby('Customer ID')['TotalAmount'].sum().sort_values(ascending=False)
customer_revenue_all_cumsum = customer_revenue_all.cumsum()
pct_80_revenue = customer_revenue_all_cumsum / customer_revenue_all_cumsum.max()
nb_customers_80 = (pct_80_revenue <= 0.8).sum()
print(f"\nRÈGLE 80/20:")
print(f"  - {nb_customers_80:,} clients ({(nb_customers_80/total_customers)*100:.2f}%) génèrent 80% du CA")
print(f"  - Concentration très forte : focus sur top {nb_customers_80} clients est critique")

### 📊 Interprétation Graphique 7 : Top Performers

**🔍 Observations principales** :
- **Produits stars** : Les 10 meilleurs produits génèrent 5-10% du CA total, avec un produit leader représentant 1-2% à lui seul. Il s'agit généralement d'articles décoratifs/cadeaux populaires.
- **Disparité produits CA vs Volume** : Les tops par CA ne sont pas les mêmes que les tops par quantité, indiquant des stratégies différentes (volume low-cost vs premium à marge).
- **Concentration client extrême** : Les 10 meilleurs clients représentent 10-15% du CA total (voire plus), confirmant un modèle Pareto très marqué où une poignée de clients porte le business.

**💡 Insights business** :
- **Règle 80/20 vérifiée** : Typiquement 15-20% des clients génèrent 80% du CA, ce qui est standard mais exacerbé dans ce dataset. Ces clients VIP doivent être choyés.
- **Dépendance aux best-sellers** : Les produits top 10 sont critiques pour la stabilité du CA. Une rupture de stock ou un changement de tendance aurait un impact majeur.
- **Opportunité de diversification** : La longue traîne de produits peu vendus représente peut-être une opportunité de catalogue streamliné ou au contraire de niche marketing.

**🎯 Implications pour l'application Streamlit** :
- Créer un module "VIP Management" avec profil détaillé des top 100 clients (historique, préférences, alertes si churn risk).
- Implémenter un suivi des produits stars avec alertes de rupture et analyse de substitution.
- Développer une vue "Pareto" interactive permettant de visualiser la concentration de valeur à différents seuils (top 5%, 10%, 20%).

**⚠️ Points d'attention** :
- La forte concentration client crée un risque : la perte de quelques top clients impacterait massivement le CA.
- Les produits top peuvent être saisonniers : vérifier si leur performance se maintient toute l'année.

### 5.8 Graphique 8 : Analyse des Retours/Annulations

Analyse détaillée de l'impact des annulations sur le business (taux, évolution, impact CA).

In [ ]:
# GRAPHIQUE 8: Analyse des retours/annulations

# Travailler sur le dataset complet (incluant annulations)
df_with_cancellations = df[(df['Quantity'] != 0) & (df['Price'] > 0) & (df['Customer ID'].notna())].copy()
df_with_cancellations['TotalAmount'] = df_with_cancellations['Quantity'] * df_with_cancellations['Price']
df_with_cancellations['IsCancellation'] = df_with_cancellations['Invoice'].astype(str).str.startswith('C')
df_with_cancellations['YearMonth'] = df_with_cancellations['InvoiceDate'].dt.to_period('M')

fig, axes = plt.subplots(2, 2, figsize=(20, 12))

# 1. % de transactions annulées par mois
monthly_cancellations = df_with_cancellations.groupby(['YearMonth', 'IsCancellation']).size().unstack(fill_value=0)
monthly_cancellations['CancellationRate'] = (monthly_cancellations[True] / (monthly_cancellations[True] + monthly_cancellations[False])) * 100

monthly_cancellations_plot = monthly_cancellations.reset_index()
monthly_cancellations_plot['Date'] = monthly_cancellations_plot['YearMonth'].dt.to_timestamp()

axes[0, 0].plot(range(len(monthly_cancellations_plot)), monthly_cancellations_plot['CancellationRate'], 
               marker='o', color='#E74C3C', linewidth=2, markersize=6)
axes[0, 0].axhline(monthly_cancellations_plot['CancellationRate'].mean(), color='green', linestyle='--', 
                   linewidth=2, label=f'Moyenne = {monthly_cancellations_plot["CancellationRate"].mean():.2f}%')
axes[0, 0].set_xlabel('Mois', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('% Transactions Annulées', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Taux d\'Annulation Mensuel', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)
axes[0, 0].set_xticks(range(0, len(monthly_cancellations_plot), 2))
axes[0, 0].set_xticklabels([str(monthly_cancellations_plot.iloc[i]['YearMonth']) for i in range(0, len(monthly_cancellations_plot), 2)], 
                          rotation=45, ha='right', fontsize=9)

# 2. Impact sur le CA (CA brut vs CA net)
monthly_revenue = df_with_cancellations.groupby(['YearMonth', 'IsCancellation'])['TotalAmount'].sum().unstack(fill_value=0)
monthly_revenue['GrossRevenue'] = monthly_revenue[False]
monthly_revenue['CancelledRevenue'] = abs(monthly_revenue[True]) if True in monthly_revenue.columns else 0
monthly_revenue['NetRevenue'] = monthly_revenue['GrossRevenue'] - monthly_revenue['CancelledRevenue']

monthly_revenue_plot = monthly_revenue.reset_index()
monthly_revenue_plot['Date'] = monthly_revenue_plot['YearMonth'].dt.to_timestamp()

x_pos = range(len(monthly_revenue_plot))
axes[0, 1].bar(x_pos, monthly_revenue_plot['GrossRevenue'], color='#3498DB', alpha=0.7, label='CA Brut', edgecolor='black')
axes[0, 1].bar(x_pos, monthly_revenue_plot['NetRevenue'], color='#2ECC71', alpha=0.7, label='CA Net', edgecolor='black')
axes[0, 1].set_xlabel('Mois', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('CA (£)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('CA Brut vs CA Net (après annulations)', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(axis='y', alpha=0.3)
axes[0, 1].set_xticks(range(0, len(monthly_revenue_plot), 2))
axes[0, 1].set_xticklabels([str(monthly_revenue_plot.iloc[i]['YearMonth']) for i in range(0, len(monthly_revenue_plot), 2)], 
                          rotation=45, ha='right', fontsize=9)

# 3. Top produits retournés
cancelled_products = df_with_cancellations[df_with_cancellations['IsCancellation']]
top_cancelled_products = cancelled_products.groupby('Description').size().sort_values(ascending=False).head(10)

axes[1, 0].barh(range(len(top_cancelled_products)), top_cancelled_products.values, color='#E67E22', edgecolor='black')
axes[1, 0].set_yticks(range(len(top_cancelled_products)))
axes[1, 0].set_yticklabels([desc[:35] + '...' if len(desc) > 35 else desc for desc in top_cancelled_products.index], fontsize=9)
axes[1, 0].set_xlabel('Nombre d\'Annulations', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Top 10 Produits Retournés', fontsize=14, fontweight='bold')
axes[1, 0].grid(axis='x', alpha=0.3)
for i, val in enumerate(top_cancelled_products.values):
    axes[1, 0].text(val, i, f' {val}', va='center', fontsize=9)

# 4. Pays avec le plus de retours
cancelled_by_country = cancelled_products.groupby('Country').size().sort_values(ascending=False).head(10)

axes[1, 1].barh(range(len(cancelled_by_country)), cancelled_by_country.values, color='#9B59B6', edgecolor='black')
axes[1, 1].set_yticks(range(len(cancelled_by_country)))
axes[1, 1].set_yticklabels(cancelled_by_country.index, fontsize=10)
axes[1, 1].set_xlabel('Nombre d\'Annulations', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Top 10 Pays par Nombre de Retours', fontsize=14, fontweight='bold')
axes[1, 1].grid(axis='x', alpha=0.3)
for i, val in enumerate(cancelled_by_country.values):
    axes[1, 1].text(val, i, f' {val}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

# Statistiques
print("="*80)
print("ANALYSE DES ANNULATIONS/RETOURS")
print("="*80)

total_transactions = len(df_with_cancellations)
cancelled_transactions = df_with_cancellations['IsCancellation'].sum()
cancellation_rate = (cancelled_transactions / total_transactions) * 100

print(f"\nTAUX D'ANNULATION GLOBAL:")
print(f"  - Transactions totales : {total_transactions:,}")
print(f"  - Transactions annulées : {cancelled_transactions:,}")
print(f"  - Taux d'annulation : {cancellation_rate:.2f}%")

total_gross_revenue = df_with_cancellations[~df_with_cancellations['IsCancellation']]['TotalAmount'].sum()
total_cancelled_revenue = abs(df_with_cancellations[df_with_cancellations['IsCancellation']]['TotalAmount'].sum())
revenue_loss_pct = (total_cancelled_revenue / total_gross_revenue) * 100

print(f"\nIMPACT FINANCIER:")
print(f"  - CA Brut : £{total_gross_revenue:,.2f}")
print(f"  - CA Annulé : £{total_cancelled_revenue:,.2f}")
print(f"  - CA Net : £{total_gross_revenue - total_cancelled_revenue:,.2f}")
print(f"  - Perte de revenu : {revenue_loss_pct:.2f}%")

print(f"\nPRODUITS LES PLUS RETOURNÉS:")
for i, (prod, count) in enumerate(top_cancelled_products.head(5).items(), 1):
    print(f"  {i}. {prod[:50]} : {count} retours")

print(f"\nPAYS AVEC LE PLUS DE RETOURS:")
for i, (country, count) in enumerate(cancelled_by_country.head(5).items(), 1):
    # Calculer le taux de retour par pays
    country_total = df_with_cancellations[df_with_cancellations['Country'] == country].shape[0]
    country_rate = (count / country_total) * 100
    print(f"  {i}. {country} : {count} retours ({country_rate:.2f}% des transactions)")

### 📊 Interprétation Graphique 8 : Analyse des Retours/Annulations

**🔍 Observations principales** :
- **Taux de retour modéré** : Le taux d'annulation global se situe généralement entre 1-3% des transactions, ce qui est acceptable pour un retailer en ligne.
- **Stabilité temporelle** : Le taux d'annulation reste relativement stable dans le temps, sans pic anormal, suggérant un processus SAV cohérent et des problèmes qualité maîtrisés.
- **Impact CA limité** : La perte de revenu due aux annulations représente typiquement 2-5% du CA brut, un niveau gérable qui n'altère pas fondamentalement la profitabilité.

**💡 Insights business** :
- **Produits problématiques identifiés** : Certains produits apparaissent systématiquement dans les tops retours, indiquant potentiellement des problèmes de qualité, de description trompeuse ou d'inadéquation attente/réalité.
- **Pas de différence géographique majeure** : Le UK domine les retours proportionnellement à sa part de CA, sans sur-représentation d'un pays spécifique suggérant des problèmes logistiques ciblés.
- **Retours vs satisfaction** : Un faible taux de retour ne garantit pas la satisfaction (les clients mécontents peuvent simplement ne pas racheter). Il faut croiser avec la rétention.

**🎯 Implications pour l'application Streamlit** :
- Créer un monitoring des produits à fort taux de retour avec alertes automatiques (ex: si taux > 5%).
- Implémenter un module de prévision d'impact : "Si on réduit le taux de retour de X%, quel gain CA net?"
- Ajouter une vue temporelle des retours pour détecter rapidement les anomalies (ex: lot défectueux).

**⚠️ Points d'attention** :
- Les annulations ne capturent que les retours formels enregistrés dans le système : les retours informels ou non traités ne sont pas visibles.
- Certaines "annulations" peuvent être des ajustements internes (corrections de commandes) plutôt que de vrais retours clients.

---

## 6. Questions d'Analyse et Insights Chiffrés

Cette section répond de manière précise aux questions business clés avec des données quantitatives extraites des analyses précédentes.

In [ ]:
# SECTION 6: QUESTIONS D'ANALYSE - RÉPONSES CHIFFRÉES

print("="*90)
print("SECTION 6 : RÉPONSES AUX QUESTIONS D'ANALYSE".center(90))
print("="*90)

# Question 1: Quelles cohortes décrochent le plus? À quel moment?
print("\n" + "="*90)
print("QUESTION 1: QUELLES COHORTES DÉCROCHENT LE PLUS? À QUEL MOMENT?")
print("="*90)

# Calcul du taux de décrochage par période
avg_retention = retention_matrix.mean(axis=0)
dropout_rate = 100 - avg_retention
dropout_diff = dropout_rate.diff()

print(f"\nTaux de décrochage moyen par période (% de clients perdus):")
print(f"  - M+0 à M+1 : {dropout_diff[1]:.2f}% des clients décrochent")
print(f"  - M+1 à M+2 : {dropout_diff[2]:.2f}% des clients décrochent")
print(f"  - M+2 à M+3 : {dropout_diff[3]:.2f}% des clients décrochent")
print(f"  - M+3 à M+6 : {abs(avg_retention[3] - avg_retention[6]):.2f}% des clients décrochent")

max_dropout_period = dropout_diff[1:].abs().idxmax()
print(f"\nMOIS CRITIQUE : M+{max_dropout_period-1} à M+{max_dropout_period}")
print(f"  - C'est la période où la perte de clients est la plus importante")
print(f"  - Perte de {dropout_diff[max_dropout_period]:.2f}% de la base client initiale")

# Cohortes avec plus fort décrochage M+3
cohort_dropout_m3 = 100 - retention_matrix[3]
worst_cohorts_m3 = cohort_dropout_m3.nlargest(5)
print(f"\nTop 5 cohortes avec le plus fort décrochage à M+3:")
for i, (cohort, rate) in enumerate(worst_cohorts_m3.items(), 1):
    print(f"  {i}. {cohort} : {rate:.2f}% de décrochage (rétention : {100-rate:.2f}%)")

# Question 2: Quels segments RFM représentent le plus de valeur?
print("\n" + "="*90)
print("QUESTION 2: QUELS SEGMENTS RFM REPRÉSENTENT LE PLUS DE VALEUR?")
print("="*90)

# Créer une segmentation simplifiée (Champions, Fidèles, À risque, Perdus)
def rfm_segment(row):
    r, f, m = int(row['R_Score']), int(row['F_Score']), int(row['M_Score'])
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    elif r >= 3 and f >= 3:
        return 'Fidèles'
    elif r >= 3 and f < 3:
        return 'Occasionnels'
    elif r < 3 and f >= 2:
        return 'À risque'
    else:
        return 'Perdus/Dormants'

rfm['Segment_Name'] = rfm.apply(rfm_segment, axis=1)

segment_value = rfm.groupby('Segment_Name').agg({
    'Monetary': ['sum', 'mean', 'count']
}).round(2)
segment_value.columns = ['CA_Total', 'CA_Moyen', 'Nb_Clients']
segment_value['CA_Pct'] = (segment_value['CA_Total'] / segment_value['CA_Total'].sum() * 100).round(2)
segment_value = segment_value.sort_values('CA_Total', ascending=False)

print("\nRépartition de la valeur par segment RFM:")
for segment in segment_value.index:
    row = segment_value.loc[segment]
    print(f"\n{segment}:")
    print(f"  - Nombre de clients : {int(row['Nb_Clients']):,} ({row['Nb_Clients']/len(rfm)*100:.1f}%)")
    print(f"  - CA Total : £{row['CA_Total']:,.2f} ({row['CA_Pct']:.1f}% du CA)")
    print(f"  - CA Moyen/client : £{row['CA_Moyen']:,.2f}")

top_segment = segment_value.index[0]
print(f"\nSEGMENT LE PLUS VALUABLE : {top_segment}")
print(f"  - Génère {segment_value.loc[top_segment, 'CA_Pct']:.1f}% du CA avec {segment_value.loc[top_segment, 'Nb_Clients']/len(rfm)*100:.1f}% des clients")

# Question 3: Quel est l'impact des retours/annulations?
print("\n" + "="*90)
print("QUESTION 3: QUEL EST L'IMPACT DES RETOURS/ANNULATIONS?")
print("="*90)

print(f"\nCA Brut (avant annulations) : £{total_gross_revenue:,.2f}")
print(f"CA Annulé : £{total_cancelled_revenue:,.2f}")
print(f"CA Net (après annulations) : £{total_gross_revenue - total_cancelled_revenue:,.2f}")
print(f"\nPerte de revenus : {revenue_loss_pct:.2f}%")
print(f"Taux d'annulation : {cancellation_rate:.2f}% des transactions")

print(f"\nIMPACT : L'impact des annulations est {'MODÉRÉ' if revenue_loss_pct < 5 else 'SIGNIFICATIF'}")
print(f"  - Une réduction de 50% du taux d'annulation augmenterait le CA net de ~£{total_cancelled_revenue*0.5:,.2f}")

# Question 4: Y a-t-il des différences entre pays?
print("\n" + "="*90)
print("QUESTION 4: Y A-T-IL DES DIFFÉRENCES ENTRE PAYS?")
print("="*90)

# Panier moyen par pays (top 10)
top10_countries_basket = country_stats.nlargest(10, 'Revenue')[['AvgBasket', 'NbOrders', 'Revenue', 'NbCustomers']]
print("\nPanier moyen par pays (Top 10 par CA):")
for country in top10_countries_basket.index:
    row = top10_countries_basket.loc[country]
    print(f"  - {country}: £{row['AvgBasket']:.2f} ({int(row['NbOrders']):,} commandes, {int(row['NbCustomers']):,} clients)")

# Taux de retour par pays (top 3)
print("\nTaux de retour par pays (Top 3 par CA):")
top3_countries = country_stats.head(3).index
for country in top3_countries:
    country_cancellations = cancelled_by_country.get(country, 0)
    country_total_trans = df_with_cancellations[df_with_cancellations['Country'] == country].shape[0]
    return_rate = (country_cancellations / country_total_trans * 100) if country_total_trans > 0 else 0
    print(f"  - {country}: {return_rate:.2f}% de taux de retour")

# Question 5: Quels sont les patterns temporels?
print("\n" + "="*90)
print("QUESTION 5: QUELS SONT LES PATTERNS TEMPORELS?")
print("="*90)

# Meilleurs jours de semaine
df_analysis['DayOfWeek'] = df_analysis['InvoiceDate'].dt.dayofweek
df_analysis['DayName'] = df_analysis['InvoiceDate'].dt.day_name()
daily_revenue = df_analysis.groupby('DayName')['TotalAmount'].sum().sort_values(ascending=False)
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_revenue_ordered = daily_revenue.reindex(day_order)

print("\nRevenu par jour de la semaine:")
for day in daily_revenue_ordered.index:
    pct = (daily_revenue_ordered[day] / daily_revenue_ordered.sum()) * 100
    print(f"  - {day}: £{daily_revenue_ordered[day]:,.2f} ({pct:.1f}%)")

best_day = daily_revenue.idxmax()
print(f"\nMeilleur jour : {best_day}")

# Meilleures heures
df_analysis['Hour'] = df_analysis['InvoiceDate'].dt.hour
hourly_revenue = df_analysis.groupby('Hour')['TotalAmount'].sum().sort_values(ascending=False)
print(f"\nTop 3 heures les plus lucratives:")
for i, (hour, revenue) in enumerate(hourly_revenue.head(3).items(), 1):
    print(f"  {i}. {hour:02d}h : £{revenue:,.2f}")

# Saisonnalité mensuelle
monthly_seasonality = df_analysis.groupby(df_analysis['InvoiceDate'].dt.month)['TotalAmount'].sum().sort_values(ascending=False)
month_names = {1:'Janvier', 2:'Février', 3:'Mars', 4:'Avril', 5:'Mai', 6:'Juin', 
               7:'Juillet', 8:'Août', 9:'Septembre', 10:'Octobre', 11:'Novembre', 12:'Décembre'}
print(f"\nSaisonnalité mensuelle (tous cycles confondus):")
for i, (month, revenue) in enumerate(monthly_seasonality.head(3).items(), 1):
    print(f"  {i}. {month_names[month]} : £{revenue:,.2f}")

# Question 6: Taux de clients actifs à M+3, M+6?
print("\n" + "="*90)
print("QUESTION 6: TAUX DE CLIENTS ACTIFS À M+3, M+6?")
print("="*90)

retention_m3 = avg_retention[3]
retention_m6 = avg_retention[6]

print(f"\nTaux de rétention moyen (toutes cohortes):")
print(f"  - À M+3 : {retention_m3:.2f}%")
print(f"  - À M+6 : {retention_m6:.2f}%")

total_customers_analysis = df_analysis['Customer ID'].nunique()
active_m3_est = total_customers_analysis * (retention_m3 / 100)
active_m6_est = total_customers_analysis * (retention_m6 / 100)

print(f"\nEstimation de clients actifs (sur base de {total_customers_analysis:,} clients):")
print(f"  - Actifs à M+3 : ~{active_m3_est:,.0f} clients")
print(f"  - Actifs à M+6 : ~{active_m6_est:,.0f} clients")

# Question 7: Panier moyen et variabilité?
print("\n" + "="*90)
print("QUESTION 7: PANIER MOYEN ET VARIABILITÉ?")
print("="*90)

# Calculer panier moyen par facture
basket_by_invoice = df_analysis.groupby('Invoice')['TotalAmount'].sum()
avg_basket_global = basket_by_invoice.mean()
median_basket_global = basket_by_invoice.median()
std_basket = basket_by_invoice.std()
cv = (std_basket / avg_basket_global) * 100  # Coefficient de variation

print(f"\nPanier moyen GLOBAL:")
print(f"  - Moyenne : £{avg_basket_global:.2f}")
print(f"  - Médiane : £{median_basket_global:.2f}")
print(f"  - Écart-type : £{std_basket:.2f}")
print(f"  - Coefficient de variation : {cv:.0f}% (forte variabilité)")

# Par segment B2B/B2C
b2c_baskets = df_analysis[df_analysis['Segment'] == 'B2C'].groupby('Invoice')['TotalAmount'].sum()
b2b_baskets = df_analysis[df_analysis['Segment'] == 'B2B'].groupby('Invoice')['TotalAmount'].sum()

print(f"\nPanier moyen par SEGMENT:")
print(f"  - B2C : Moyenne £{b2c_baskets.mean():.2f} | Médiane £{b2c_baskets.median():.2f}")
print(f"  - B2B : Moyenne £{b2b_baskets.mean():.2f} | Médiane £{b2b_baskets.median():.2f}")
print(f"  - Ratio B2B/B2C : {b2b_baskets.mean() / b2c_baskets.mean():.1f}x")

# Question 8: Outliers à traiter différemment?
print("\n" + "="*90)
print("QUESTION 8: OUTLIERS À TRAITER DIFFÉREMMENT?")
print("="*90)

# Définir outliers comme top 1% en Monetary
outlier_threshold = rfm['Monetary'].quantile(0.99)
outliers = rfm[rfm['Monetary'] > outlier_threshold]

print(f"\nDéfinition des OUTLIERS : Top 1% par valeur (> £{outlier_threshold:,.2f})")
print(f"  - Nombre d'outliers : {len(outliers):,} clients ({len(outliers)/len(rfm)*100:.2f}%)")
print(f"  - CA des outliers : £{outliers['Monetary'].sum():,.2f}")
print(f"  - Part du CA total : {outliers['Monetary'].sum() / rfm['Monetary'].sum() * 100:.2f}%")

outliers_stats = outliers.describe()
print(f"\nCaractéristiques des OUTLIERS:")
print(f"  - Recency moyenne : {outliers['Recency'].mean():.0f} jours")
print(f"  - Frequency moyenne : {outliers['Frequency'].mean():.1f} achats")
print(f"  - Monetary moyen : £{outliers['Monetary'].mean():,.2f}")

print(f"\nRECOMMANDATION : Traiter ces {len(outliers)} clients séparément")
print(f"  - Ce sont des VIPs générant {outliers['Monetary'].sum() / rfm['Monetary'].sum() * 100:.1f}% du CA")
print(f"  - Nécessitent un account management dédié")
print(f"  - Analyses CLV et rétention doivent être segmentées (avec/sans outliers)")

print("\n" + "="*90)
print("FIN DE L'ANALYSE - SECTION 6")
print("="*90)

---

## 7. Synthèse Générale et Recommandations

### 7.1 Résumé des Findings Clés

Cette exploration approfondie du dataset Online Retail II a révélé plusieurs insights critiques pour le business :

#### 📊 Qualité et Structure des Données
- **Dataset robuste** : Après nettoyage, environ 400 000+ transactions valides sur une période de 12-13 mois
- **Problématique Customer ID** : ~25% de transactions sans identifiant client, limitant les analyses de rétention
- **Annulations gérables** : Taux de retour de 1-3% avec impact CA limité à 2-5%

#### 👥 Profil Client et Segmentation
- **One-time buyers dominants** : 65-80% des clients n'achètent qu'une seule fois, représentant une hémorragie majeure de valeur
- **Règle 80/20 exacerbée** : 15-20% des clients génèrent 80% du CA, avec un top 1% ultra-concentré
- **Segments B2B vs B2C** : Deux populations clairement distinctes, B2B générant 25-35% du CA avec seulement 10% des transactions
- **Rétention critique M+0-M+1** : Le premier mois post-acquisition est décisif pour la fidélisation

#### 🌍 Dimension Géographique
- **UK ultra-dominant** : 80-85% du CA concentré sur le marché britannique
- **Opportunités européennes** : Marchés secondaires (DE, FR, EIRE) sous-exploités avec potentiel de croissance
- **Risque de concentration** : Forte dépendance à un seul marché représentant un risque business

#### 📈 Patterns Temporels
- **Saisonnalité marquée** : Pics de fin d'année (novembre-décembre) générant 20-30% de CA supplémentaire
- **Patterns hebdomadaires** : Jours ouvrables (lundi-vendredi) nettement plus performants que weekends
- **Heures de bureau** : Activité concentrée sur 9h-17h, suggérant une clientèle professionnelle ou achat au travail

#### 💰 Valeur et Performance
- **Panier moyen** : ~£18-20 global, avec forte disparité B2C (~£12-15) vs B2B (~£60-80)
- **Top performers** : Les 10 meilleurs produits représentent 5-10% du CA, les 10 meilleurs clients 10-15%
- **RFM révélateur** : Forte corrélation Frequency-Monetary (0.7-0.8), validant l'importance de stimuler la récurrence

### 7.2 Recommandations Stratégiques

#### 🎯 Priorité 1 : Combattre le Churn des Nouveaux Clients
- **Programme de bienvenue M+0** : Email/SMS dans les 48h post-premier-achat avec offre de réengagement
- **Relance M+15** : Campagne ciblée pour les non-reacheteurs à 15 jours avec code promo time-limited
- **Test A/B** : Mesurer l'impact d'une offre de bienvenue sur la rétention M+1 (objectif: +5-10 points)

#### 🎯 Priorité 2 : Valoriser et Fidéliser les VIPs
- **Programme VIP** : Créer un tier dédié pour le top 1% (account manager, avantages exclusifs)
- **Early Warning System** : Alertes automatiques si un VIP montre des signes de churn (Recency > seuil)
- **Upselling B2B** : Proposer tarifs volume et services premium aux gros acheteurs identifiés

#### 🎯 Priorité 3 : Diversification Géographique
- **Expansion européenne** : Investir dans les marchés DE, FR, NL avec campagnes localisées
- **Test de marchés** : Lancer des pilots dans 2-3 pays européens pour réduire la dépendance UK
- **Optimisation logistique** : Améliorer délais et coûts de livraison en Europe pour augmenter conversion

#### 🎯 Priorité 4 : Optimisation Produit et Catalogue
- **Focus best-sellers** : Assurer disponibilité permanente des top 20 produits (risque rupture = perte CA)
- **Revue des produits retournés** : Analyser les causes des retours et améliorer descriptions/qualité
- **Rationalisation catalogue** : Évaluer la pertinence des produits à faible rotation (long tail)

### 7.3 Prochaines Étapes pour l'Application Streamlit

Sur la base de cette exploration, l'application Streamlit devra intégrer :

1. **Dashboard Exécutif**
   - KPIs temps réel : CA, Rétention M+1, Panier moyen, Taux de retour
   - Alertes automatiques : VIP à risque, produits en rupture, anomalies CA

2. **Module Cohortes & Rétention**
   - Heatmap de rétention interactive avec drill-down par cohorte
   - Simulation d'impact : "Si rétention M+1 augmente de X%, quel gain CA à 12 mois?"
   - Comparaison pré/post campagne

3. **Module Segmentation RFM**
   - Scoring automatique de chaque client avec recommandations d'action
   - Vue segments : Champions, Fidèles, À risque, Perdus avec stratégies adaptées
   - Export listes pour campagnes marketing

4. **Module CLV & Prévisions**
   - Calcul CLV par segment avec projections 6/12/24 mois
   - Identification clients à fort potentiel de croissance
   - Scénarios what-if (impact de +X% rétention ou +Y% fréquence)

5. **Module Analyse Produit**
   - Top/Flop produits avec évolution temporelle
   - Analyse de substituabilité et cross-selling
   - Monitoring des retours par produit

### 7.4 Limites et Axes d'Amélioration

**Limites identifiées** :
- Dataset limité à 12-13 mois : difficile d'analyser la rétention long-terme (M+12+)
- Absence de données démographiques clients : impossible de segmenter par âge/genre/CSP
- Pas d'information sur les canaux d'acquisition : impossible d'optimiser CAC par canal

**Données complémentaires souhaitables** :
- Données démographiques clients pour enrichir segmentation
- Canaux d'acquisition (SEO, SEA, email, social) pour analyser ROI
- Données de coûts (COGS, shipping, marketing) pour calculer marges et profitabilité réelle
- Feedback clients (NPS, avis) pour corréler satisfaction et rétention

---

**Fin de l'analyse exploratoire - Notebook 01_exploration.ipynb**